# Product Line Selection

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import random
import gurobipy as gp
from gurobipy import GRB
import warnings
warnings.filterwarnings('ignore') # setting ignore as a parameter
import math
from datetime import datetime
import openpyxl

C:\Users\afons\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\afons\anaconda3\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\afons\anaconda3\lib\site-packages\numpy\.libs\libopenblas.gk7gx5keq4f6uyo3p26ulgbqyhgqo7j4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Functions

In [19]:
#Isto aqui so exporta para excel os dados da instance para automatizar ver restrições
def export_to_excel(data, filename):
   
    n_periods, n_products, n_categories, n_families, margins, holding_costs, setup_costs, setup_times, attraction, fix_costs, res_utilization, market_size, capacity, shares, competition, fam_setup_times, fam_setup_costs, products_by_fam, products_by_cat = data
    
    
    with pd.ExcelWriter(filename) as writer:
        pd.DataFrame(margins).to_excel(writer, sheet_name='margins')
        pd.DataFrame(holding_costs).to_excel(writer, sheet_name='holding_costs')
        pd.DataFrame(setup_costs).to_excel(writer, sheet_name='setup_costs')
        pd.DataFrame(setup_times).to_excel(writer, sheet_name='setup_times')
        pd.DataFrame(attraction).to_excel(writer, sheet_name='attraction')
        pd.DataFrame(fix_costs).to_excel(writer, sheet_name='fix_costs')
        pd.DataFrame(res_utilization).to_excel(writer, sheet_name='res_utilization')
        pd.DataFrame(market_size).to_excel(writer, sheet_name='market_size')
        pd.DataFrame(capacity).to_excel(writer, sheet_name='capacity')
        pd.DataFrame(competition).to_excel(writer, sheet_name='competition')
        pd.DataFrame(fam_setup_times).to_excel(writer, sheet_name='fam_setup_times')
        pd.DataFrame(fam_setup_costs).to_excel(writer, sheet_name='fam_setup_costs')
        pd.DataFrame(products_by_fam).to_excel(writer, sheet_name='products_by_fam')

#Ler ficheiros unsolved e solved
def read_file_unsolved(instance):
    with open(instance, mode='r', encoding='utf-8') as file:
        data = file.readlines()
        n_periods = int(data[0])
        n_products = int(data[1])
        n_categories = int(data[2])
        n_families = int(data[3])
        
        #margins
        aux = data[4]
        aux = aux.split(' ')
        aux = aux[:-1]
        margins = [float(i) for i in aux]

        #holding costs
        aux = data[6]
        aux = aux.split(' ')
        aux = aux[:-1]
        holding_costs = [float(i) for i in aux]
        
        #setup costs
        aux = data[8]
        aux = aux.split(' ')
        aux = aux[:-1]
        setup_costs = [float(i) for i in aux]
        
        #setup times
        aux = data[10]
        aux = aux.split(' ')
        aux = aux[:-1]
        setup_times = [float(i) for i in aux]
        
        #attraction
        aux = data[12]
        aux = aux.split(' ')
        aux = aux[:-1]
        attraction = [float(i) for i in aux]
        
        #fix_costs
        aux = data[14]
        aux = aux.split(' ')
        aux = aux[:-1]
        fix_costs = [float(i) for i in aux]
        
        #resource_utilization        
        aux = data[16]
        aux = aux.split(' ')
        aux = aux[:-1]
        res_utilization = [float(i) for i in aux]
        
        #market_size
        aux = data[18]
        aux = aux.split(' ')
        aux = aux[:-1]
        market_size = [float(i) for i in aux]
        
        #capacity       
        aux = data[20]
        aux = aux.split(' ')
        aux = aux[:-1]
        capacity = [float(i) for i in aux]

        #share
        shares=np.empty([n_periods,n_categories])
        for i in range(0,n_periods):
            aux = data[22+i]
            aux = aux.split(' ')
            aux = aux[:-1]
            aux = np.array([float(i) for i in aux])
            shares[i,]= aux
        
        #competition      
        aux = data[35]
        aux = aux.split(' ')
        aux = aux[:-1]
        competition = [float(i) for i in aux]
        
        #family set-up time
        aux = data[37]
        aux = aux.split(' ')
        aux = aux[:-1]
        fam_setup_times = [float(i) for i in aux]
        
        #family set-up costs same
        aux = data[39]
        aux = aux.split(' ')
        aux = aux[:-1]
        fam_setup_costs = [float(i) for i in aux]
        
        #family products
        products_by_fam= list()
        for i in range(n_families):
            aux = data[41+i]
            aux = aux.split(' ')
            aux = aux[:-1]
            aux2= [int(i) for i in aux]
            products_by_fam.append(aux2)
            
        #category products
        products_by_cat= list()
        for i in range(n_categories):
            aux = data[41+n_families+1+i]
            aux = aux.split(' ')
            aux = aux[:-1]
            aux2= [int(i) for i in aux]
            products_by_cat.append(aux2)

                #0           1        2         3          4            5               6              7       8                9            10       11           12               13             14          15 fixed          16 fixed         17       18
    return  n_periods, n_products, n_families, margins, holding_costs, setup_costs, setup_times, fix_costs, res_utilization, market_size, capacity, competition, fam_setup_times, fam_setup_costs, shares, products_by_fam, products_by_cat, attraction, n_categories

#Ler ficheiro real
def read_file(instance):
    with open(instance, mode='r', encoding='utf-8') as file:
        data = file.readlines()
        n_periods = int(data[0])
        n_products = int(data[1])
        n_categories = int(data[2])
        n_families = int(data[3])
        
        #margins
        aux = data[4]
        aux = aux.split('\t')
        size = len(aux[n_products-1])
        aux[n_products-1] = aux[n_products-1][:size-1]
        margins = [float(i) for i in aux]
        
        #holding costs
        aux = data[6]
        aux = aux.split('\t')
        size = len(aux[n_products-1])
        aux[n_products-1] = aux[n_products-1][:size-1]
        holding_costs = [float(i) for i in aux]
        
        #setup costs
        aux = data[8]
        aux = aux.split('\t')
        size = len(aux[n_products-1])
        aux[n_products-1] = aux[n_products-1][:size-1]
        setup_costs = [float(i) for i in aux]
        
        #setup times
        aux = data[10]
        aux = aux.split('\t')
        size = len(aux[n_products-1])
        aux[n_products-1] = aux[n_products-1][:size-1]
        setup_times = [float(i) for i in aux]
        
        #attraction
        aux = data[12]
        aux = aux.split('\t')
        size = len(aux[n_products-1])
        aux[n_products-1] = aux[n_products-1][:size-1]
        attraction = [float(i) for i in aux]
        
        #fix_costs
        aux = data[14]
        aux = aux.split('\t')
        size = len(aux[n_products-1])
        aux[n_products-1] = aux[n_products-1][:size-1]
        fix_costs = [float(i) for i in aux]
        
        #resource_utilization        
        aux = data[16]
        aux = aux.split('\t')
        size = len(aux[n_products-1])
        aux[n_products-1] = aux[n_products-1][:size-1]
        res_utilization = [float(i) for i in aux]
        
        #market_size
        aux = data[18]
        aux = aux.split('\t')
        size = len(aux[n_periods-1])
        aux[n_periods-1] = aux[n_periods-1][:size-1]
        market_size = [float(i) for i in aux]
        
        #capacity       
        aux = data[20]
        aux = aux.split('\t')
        size = len(aux[n_periods-1])
        aux[n_periods-1] = aux[n_periods-1][:size-1]
        capacity = [float(i) for i in aux]
        
        #share
        shares=np.empty([n_periods,n_categories])
        for i in range(0,n_periods):
            aux = data[22+i]
            aux = aux.split('\t')
            size = len(aux[n_categories-1])
            aux[n_categories-1] = aux[n_categories-1][:size-1]
            aux = np.array([float(i) for i in aux])
            shares[i,]= aux
        
        #competition      
        aux = data[35]
        aux = aux.split('\t')
        size = len(aux[n_families-1])
        aux[n_families-1] = aux[n_families-1][:size-1]
        competition = [float(i) for i in aux]
        
        #family set-up time
        aux = data[37]
        aux = aux.split('\t')
        size = len(aux[n_families-1])
        aux[n_families-1] = aux[n_families-1][:size-1]
        fam_setup_times = [float(i) for i in aux]
        
        #family set-up costs same
        aux = data[39]
        aux = aux.split('\t')
        size = len(aux[n_families-1])
        aux[n_families-1] = aux[n_families-1][:size-1]
        fam_setup_costs = [float(i) for i in aux]
        
        #family products
        products_by_fam= list()
        for i in range(n_families):
            aux = data[41+i]
            aux = aux.split('\t')
            size = len(aux[-1])
            aux[-1] = aux[-1][:size-1]
            aux2= [int(i) for i in aux]
            products_by_fam.append(aux2)
            
        #category products
        products_by_cat= list()
        for i in range(n_categories):
            aux = data[41+n_families+1+i]
            aux = aux.split('\t')
            size = len(aux[-1])
            aux[-1] = aux[-1][:size-1]
            aux2= [int(i) for i in aux]
            products_by_cat.append(aux2)

                #0           1        2         3          4            5               6              7       8                9            10       11           12               13             14          15 fixed          16 fixed         17       18
    return  n_periods, n_products, n_families, margins, holding_costs, setup_costs, setup_times, fix_costs, res_utilization, market_size, capacity, competition, fam_setup_times, fam_setup_costs, shares, products_by_fam, products_by_cat, attraction, n_categories

#Constraints
#3.2 não é preciso
def demand_more_than_sold(Xamounts_sold, Xshare_product):
    feasible = True
    for k in range(0,len(data[16])):
        array = np.array(data[16][k])
        for j in range(0,len(array)):
            diff = Xshare_product[array[j]] * category_demand_per_period.loc[k,:] - Xamounts_sold.iloc[array[j],:]
            if sum(diff < -1) > 0:
                feasible = False  
    return feasible, diff

#3.3 não é preciso
def probability_only_exists_if_selected(Xproduct_selection, Xshare_product):
    feasible = True
    aux = Xproduct_selection- Xshare_product
    aux = aux < 0
    if aux.sum() > 0:
        feasible = False
    return feasible

#3.5
def attractiveness(Xslack_product, Xshare_competition):
    diff = np.zeros(J)
    feasible = True
    for k in range(0,len(data[16])):
        array = np.array(data[16][k])
        for j in range(0,len(array)):
            diff[array[j]] = data[17][array[j]] * Xshare_competition[k] - Xslack_product[array[j]]
    if sum(diff < 0) > 0:
        feasible = False  
    return feasible, diff

#3.6 não é preciso
def share_product_total_1(Xshare_product, Xshare_competition):
    feasible = True
    for k in range(0,len(data[16])):
        acumulado = 0
        array = np.array(data[16][k])
        acumulado = acumulado + Xshare_competition[k]
        for j in range(0,len(array)):
            acumulado = acumulado + Xshare_product[array[j]]
        if acumulado > 1.05:
            feasible = False 
        if acumulado < 0.95:
            feasible = False  
    return feasible

### Production Planning

#3.9
def production_constraint(Xproduction, Xproduct_selected):
    feasible = True
    aux = Xproduct_selected * max_production - Xproduction
    if aux.sum().sum() < 0:
        feasible = False
    return feasible, aux

#3.11 não é preciso
def exist_holding_inventory(Xamounts_sold,  Xholding_inventory, Xproduction):
    feasible = True
    lag_holding_inventory = Xholding_inventory.shift(1, axis = 1)
    lag_holding_inventory.iloc[:,0] = 0
    aux = Xamounts_sold - (lag_holding_inventory + Xproduction - Xholding_inventory)
    aux = (aux != 0)
    if aux.sum().sum() > 0:
        feasible = False
    return feasible

#3.15 (+ #3.14 alterada)
def capacity_constraint(Xproduct_selected, Xproduction, Xproduct_selected_family):
    #check capacity
    feasible = True
    aux1 = Xproduct_selected_family.T * data[12]
    aux2 = Xproduct_selected.T * data[6] + Xproduction.T * data[8]
    aux = aux1.sum(axis = 'columns') + aux2.sum(axis = 'columns')
    diff = data[10] - aux
    if sum(diff < 0) > 0:
        feasible = False
    return feasible, diff

#Evaluation function
def evaluate_function(Xamounts_sold, Xholding_inventory, Xproduct_selected, Xproduct_selected_family, Xproduct_selection):
    revenue = sum(Xamounts_sold.sum(axis = 'columns') * data[3])
    inventory_holding_cost = sum(Xholding_inventory.sum(axis = 'columns') * data[4]) 
    setup_cost = sum(Xproduct_selected.sum(axis = 'columns') * data[5])
    famility_setup_cost = sum(Xproduct_selected_family.sum(axis = 'columns') * data[13])
    fixed_cost = sum(Xproduct_selection * data[7])
    total = revenue - inventory_holding_cost - setup_cost - famility_setup_cost - fixed_cost
    return total

#Feasibility function
def verify_feasibility(Xamounts_sold, Xshare_product, Xproduct_selection, Xshare_competition, Xproduct_selected, Xproduction, Xproduct_selected_family, Xslack_product):
    feasible1 = demand_more_than_sold(Xamounts_sold, Xshare_product)
    feasible2 = probability_only_exists_if_selected(Xproduct_selection, Xshare_product)
    feasible3 = attractiveness(Xslack_product, Xshare_competition) #SIM
    feasible4 = share_product_total_1(Xshare_product, Xshare_competition)
    feasible5 = production_constraint(Xproduction, Xproduct_selected) #SIM
    feasible6 = capacity_constraint(Xproduct_selected, Xproduction, Xproduct_selected_family) #SIM
    feasible = feasible1[0] & feasible2 & feasible3[0] & feasible4 & feasible5[0] & feasible6[0]
    #print(feasible1[0])
    #print(feasible2)
    #print(feasible3[0])
    #print(feasible4)
    #print(feasible5[0])
    #print(feasible6[0])
        #overall, feasible1, over demand,     attractivenes bool, attractiveness over slack,   production bool,  excess production,   capacity bool,    over capacity (t)    
    return feasible, feasible1[0], feasible1[1], feasible3[0],       feasible3[1],             feasible5[0],     feasible5[1],      feasible6[0],      feasible6[1]

#Constructive
#Share computation for constructive
def share_computation(p_selected, j_product):
    #Modelo Shares
    share_model = gp.Model('Share_optmization')
    share_model.Params.LogToConsole = 0

    #decisionvariables
    p_share = share_model.addVars(data[1],lb=0, ub=1, vtype=GRB.CONTINUOUS) 
    k_share_competition = share_model.addVars(data[18],lb=0,ub=1, vtype=GRB.CONTINUOUS)
    p_slack= share_model.addVars(data[1],lb=0,ub=1000,vtype=GRB.CONTINUOUS)
    #p_amounts_sold_t = m.addVars(data[0], data[1], lb=0, vtype=GRB.INTEGER)

    #constraints
    c1=c2=c3=c4=c5 = list()
    
    #for i in range(0, data[0]):
        #for j in range(0, data[1]):
            #c6 = share_model.addConstr(p_amounts_sold_t[i,j] - (p_share[j])*category_demand_per_period[i][p_belongs_k(j)-1] <= 0)
 
    for i in range(0,data[1]):
        c1 = share_model.addConstr(p_share[i]<=p_selected[i])
        c5 = share_model.addConstr(p_share[i]>=product_selection[i]*0.000000000000001)
        
    for k in range(0,len(data[16])):
        array = np.array(data[16][k])
        c4.append(share_model.addConstr(k_share_competition[k] + gp.quicksum(p_share[aux] for aux in data[16][k]) == 1 ))
        for j in range(0,len(array)):
            c2.append(share_model.addConstr(p_slack[array[j]] == data[17][array[j]] * k_share_competition[k] - p_share[array[j]] * data[11][k]))
            c3.append(share_model.addConstr(p_slack[array[j]] <= data[17][array[j]] * k_share_competition[k]) )

    #objective function
    sum_shares = gp.quicksum(p_share[prod] for prod in range(0,data[1])) - gp.quicksum(k_share_competition[comp] for comp in range(0,data[18]))
    share_model.setObjective(sum_shares, GRB.MAXIMIZE)
    
    # Suppress Gurobi output by setting the LogToConsole parameter to 0
    share_model.params.LogToConsole = 0
    
    share_model.optimize()
    return p_share[j_product].X

#Obtain category of a product
def p_belongs_k(p):
    k=-1
    for i in range(0, len(data[16])):
        if p in data[16][i]:
            k=i
    return k

#Obtain family of a product
def p_belongs_m(p):
    k=-1
    for i in range(0, len(data[15])):
        if p in data[15][i]:
            m=i
    return m

#Obtain profit for a given product selected matrix (jxt)
def quantities_computation(Xproduct_selected, Xproduct_selected_family, Xproduct_selection, Xamounts_sold, Xproduction, Xholding_inventory, Xshare_product, Xshare_competition, Xslack_product):
    #model creation
    m= gp.Model('PLS')
    #Decision Variables
    p_amounts_sold_t = m.addVars(data[0], data[1], lb=0, vtype=GRB.CONTINUOUS)
    p_production_t = m.addVars(data[0],data[1],lb=0, vtype=GRB.CONTINUOUS)
    p_holding_t = m.addVars(data[0],data[1],lb=0, vtype=GRB.CONTINUOUS)
    p_share = m.addVars(data[1],lb=0, ub=1, vtype=GRB.CONTINUOUS) 
    k_share_competition = m.addVars(data[18],lb=0,ub=1, vtype=GRB.CONTINUOUS)
    p_slack= m.addVars(data[1],lb=0,ub=1000,vtype=GRB.CONTINUOUS)
    #constraints
    c1=c2=c3=c4=c5=c6=c7=c8=c9 = list()

    for i in range(0, data[0]):
        #soft - capacity
        c9.append(m.addConstr(gp.quicksum(data[12][fam]*Xproduct_selected_family[i][fam] for fam in range(0,data[2])) + gp.quicksum(data[8][prod]*Xproduct_selected[i][prod] + p_production_t[i,prod]*data[6][prod] for prod in range(0,data[1])) <= data[10][i]))
        for j in range(0, data[1]):
            c1.append( m.addConstr(p_amounts_sold_t[i,j] - (p_share[j])* category_demand_per_period[i][p_belongs_k(j)] <= 0 ))
            c7.append( m.addConstr(p_production_t[i,j] - 99999999999999 * Xproduct_selected[i][j] <= 0 ))
           # c8.append( m.addConstr(p_holding_t[i,j] - 9999999999999999 * Xholding_selected[i][j] <= 0 ))
            if i!=0:
                c6.append(m.addConstr(p_amounts_sold_t[i,j] == p_production_t[i,j] + p_holding_t[i-1,j] - p_holding_t[i,j]))
            else:
                c6.append(m.addConstr(p_amounts_sold_t[i,j] == p_production_t[i,j] - p_holding_t[i,j]))

    #3.3 c2
    for j in range(0, data[1]):
        c2.append(m.addConstr(p_share[j] <= Xproduct_selection[j]))   

    #3.4 c3
    #3.5 c4
    #3.6 c5
    for k in range(0,len(data[16])):
        array = np.array(data[16][k])
        c5.append(m.addConstr(k_share_competition[k] + gp.quicksum(p_share[aux] for aux in data[16][k]) == 1 ))
        for j in range(0,len(array)):
            c3.append(m.addConstr(p_slack[array[j]] == data[17][array[j]] * k_share_competition[k] - p_share[array[j]] * data[11][k]))
            c4.append(m.addConstr(p_slack[array[j]] <= data[17][array[j]] * k_share_competition[k]) )

    #objective function
    R = gp.quicksum(data[3][product] * p_amounts_sold_t[period,product] for period in range(0,data[0]) for product in range(0,data[1]))
    C1 = gp.quicksum(data[4][product] * p_holding_t[period,product] for period in range(0,data[0]) for product in range(0,data[1]))
    C2 = sum(Xproduct_selected.sum(axis = 'columns') * data[5])
    C3 = sum(Xproduct_selected_family.sum(axis = 'columns') * data[13])
    C4 = sum(Xproduct_selection * data[7])
    obj = R - C1 - C2 - C3 - C4
    m.setObjective(obj, GRB.MAXIMIZE)
    
    # Suppress Gurobi output by setting the LogToConsole parameter to 0
    m.params.LogToConsole = 0
    
    m.optimize()
    obj.getValue()

    for j_match in range(0,data[1]):
        for t_match in range(0,data[0]):
            Xamounts_sold[t_match][j_match] = p_amounts_sold_t[t_match,j_match].X
            Xproduction[t_match][j_match] = p_production_t[t_match,j_match].X     
            Xholding_inventory[t_match][j_match] = p_holding_t[t_match,j_match].X     
        Xshare_product[j_match] = p_share[j_match].X
        Xslack_product[j_match] = p_slack[j_match].X
        
    for k_match in range(0,K):     
        Xshare_competition[k_match] = k_share_competition[k_match].X 
        
    return Xamounts_sold, Xproduction, Xholding_inventory, Xshare_product, Xshare_competition, Xslack_product

#Get the best neighbor of a neighborhood
def best_neighbor(neighbors):
    
    Xamounts_sold = pd.DataFrame([[0]*T]*J)
    Xholding_inventory = pd.DataFrame([[0]*T]*J)
    Xproduct_selected_family = pd.DataFrame([[0]*T]*M)
    Xproduct_selection = np.zeros(J)
    Xproduct_selected = pd.DataFrame([[0]*T]*J)
    Xshare_product = np.zeros(J)
    Xshare_competition = np.zeros(K)
    Xslack_product = np.zeros(J)
    Xproduction = pd.DataFrame([[0]*T]*J)
    
    current_evaluation = 0
        
    for n in range(neighbors[12].min(), neighbors[12].max() + 1):
        new_neighbor = neighbors[neighbors[12] == n]

        new_neighbor.reset_index(inplace = True, drop = True)
        Xproduct_selected = new_neighbor.drop([12], axis=1)

        #product selection computation (substitui 3.3)
        Xproduct_selection = (Xproduct_selected.sum(axis = 'columns') > 0) * 1

        #binary setup famility computation
        for m in range(0,len(data[15])):
            Xproduct_selected_family.iloc[m,:] = Xproduct_selected.iloc[data[15][m]].sum()
            Xproduct_selected_family = (Xproduct_selected_family > 0) * 1
            
        neighbor_results = quantities_computation(Xproduct_selected, Xproduct_selected_family, Xproduct_selection, Xamounts_sold, Xproduction, Xholding_inventory, Xshare_product, Xshare_competition, Xslack_product)
        Xamounts_sold = neighbor_results[0]
        Xproduction = neighbor_results[1]
        Xholding_inventory = neighbor_results[2]
        Xshare_product  = neighbor_results[3]
        Xshare_competition = neighbor_results[4]
        Xslack_product = neighbor_results[5]

        #EVALUATE IF BETTER
        feasible_solution = verify_feasibility(Xamounts_sold, Xshare_product, Xproduct_selection, Xshare_competition, Xproduct_selected, Xproduction, Xproduct_selected_family, Xslack_product)
        new_evaluation = evaluate_function(Xamounts_sold, Xholding_inventory, Xproduct_selected, Xproduct_selected_family, Xproduct_selection)

        if ((new_evaluation > current_evaluation) & feasible_solution[0]):
            best_neighbor = Xproduct_selected
            current_evaluation = new_evaluation

    return best_neighbor, current_evaluation

# Perform a local search starting from the given initial solution with guided neighborhood structure
def local_search_guided(initial_solution, initial_evaluation):
    
    #Initialize dataframe to save results
    results = []
    best = []
    start=datetime.now()
    time = []
    
    # Initialize the best solution found so far to the initial solution
    best_solution = initial_solution
    best_evaluation = initial_evaluation
    
    # Initialize a flag to indicate if the search is complete
    finish = True
    iteration_product = 0
    N_iteration_product = 100 # maximum
    array_experimented = []

    # While the search is not complete...
    while len(array_experimented) < J:
        j = random.randint(0,J-1)
        iteration_product = iteration_product + 1
        
        # Control experimented products to ensure no repetitions
            
        while j in array_experimented:
            j = random.randint(0,J-1)
        
        array_experimented.append(j)
        
        # Generate the product neighborhood of the current solution        
        neighbors = generate_neighborhood_guided(best_solution, j)
        
        # Set the current solution to the best solution in the neighborhood
        current_solution = best_neighbor(neighbors)
        
        print(str(current_solution[1])+' / '+str(best_evaluation))
        results.append(current_solution[1])
        best.append(best_evaluation)
        time.append(datetime.now()-start)
        
        # If the current solution is better than the best solution found so far...
        if current_solution[1] > best_evaluation:
            # Update the best solution found so far
            best_solution = current_solution[0]
            best_evaluation = current_solution[1]
            array_experimented = []
    
    # Return the best solution found
    return best_solution, results, best, time

# Perform a local search starting from the given initial solution with one flip move
def local_search_blind(initial_solution, initial_evaluation):
    
    #Initialize dataframe to save results
    results = []
    best = []
    start=datetime.now()
    time = []
    new_evaluation = 0
    
    # Initialize the best solution found so far to the initial solution
    best_solution = initial_solution
    best_evaluation = initial_evaluation
    
    # Initialize a flag to indicate if the search is complete
    finish = False
    iteration_product = 0
    N_iteration_product = 100 # maximum

    # While the search is not complete...
    while not finish:
        iteration_product = iteration_product + 1
        
        # Generate the product neighborhood of the current solution        
        neighbors = generate_neighborhood_blind(best_solution)
        
        # Set the current solution to the best solution in the neighborhood
        current_solution = best_neighbor(neighbors)
        
        print(str(current_solution[1]) +' / '+ str(best_evaluation))
        results.append(current_solution[1])
        best.append(best_evaluation)
        time.append(datetime.now()-start)
        
        # If the current solution is better than the best solution found so far...
        if current_solution[1] > best_evaluation:
            # Update the best solution found so far
            best_solution = current_solution[0]
            best_evaluation = current_solution[1]
        else:
            finish = True
    
    # Return the best solution found
    return best_solution, results, best, time

# Generate guided neighborhood
def generate_neighborhood_guided(solution, j):
    
    # Initialize an empty set of neighbors
    neighbors = {0: [0], 1: [0], 2: [0], 3: [0], 4: [0], 5: [0], 6: [0], 7: [0], 8: [0], 9: [0], 10: [0], 11: [0], 12: [0]}
    neighbors = pd.DataFrame(neighbors)
    
    # Iterate over the solution and generate all possible neighboring solutions by making small, local changes
    iteration = 0
    new_solution = solution.copy()
    
    for t in range(1,T-4):
        for t2 in range(t+1,T-3):
            for t3 in range(t2+1,T-2):
                for t4 in range(t3+1,T-1):
                    for t5 in range(t4+1,T):
                        
                        ku = random.randint(0,5)
                        if ku == 1:
                        
                            iteration = iteration + 1
                            new_solution = solution.copy()
                            new_solution.iloc[j, t] = abs(solution.iloc[j, t] - 1)
                            new_solution.iloc[j, t2] = abs(solution.iloc[j, t2] - 1)
                            new_solution.iloc[j, t3] = abs(solution.iloc[j, t3] - 1)
                            new_solution.iloc[j, t4] = abs(solution.iloc[j, t4] - 1)
                            new_solution.iloc[j, t5] = abs(solution.iloc[j, t5] - 1)
                            new_solution[12] = iteration

                            # Add the new solution to the set of neighbors
                            neighbors = neighbors.append(new_solution)
    
    neighbors.reset_index(drop = True, inplace = True)
    neighbors.drop([0], axis=0, inplace=True)
    neighbors.reset_index(drop = True, inplace = True)
    # Return the set of neighboring solutions
    return neighbors

# Generate neighborhood of one flip move
def generate_neighborhood_blind(solution):
    
    # Initialize an empty set of neighbors
    neighbors = {0: [0], 1: [0], 2: [0], 3: [0], 4: [0], 5: [0], 6: [0], 7: [0], 8: [0], 9: [0], 10: [0], 11: [0], 12: [0]}
    neighbors = pd.DataFrame(neighbors)
    
    # Iterate over the solution and generate all possible neighboring solutions by making small, local changes
    iteration = 0
    new_solution = solution.copy()
    
    for t in range(0,T):
        for j in range(0,J):
            iteration = iteration + 1
            new_solution = solution.copy()
            new_solution.iloc[j, t] = abs(solution.iloc[j, t] - 1)
            new_solution[12] = iteration

            # Add the new solution to the set of neighbors
            neighbors = neighbors.append(new_solution)
    
    neighbors.reset_index(drop = True, inplace = True)
    neighbors.drop([0], axis=0, inplace=True)
    neighbors.reset_index(drop = True, inplace = True)
    # Return the set of neighboring solutions
    return neighbors

# Generate neighborhood of one flip move
def generate_neighbor_simulated_annealing(solution):
    
    new_solution = solution.copy()
    lim = random.randint(0,5)
    for k in range(1,lim+1):
        j = random.randint(0,J-1)
        t = random.randint(0,T-1)
        new_solution.iloc[j, t] = abs(solution.iloc[j, t] - 1)
    
    Xamounts_sold = pd.DataFrame([[0]*T]*J)
    Xholding_inventory = pd.DataFrame([[0]*T]*J)
    Xproduct_selected_family = pd.DataFrame([[0]*T]*M)
    Xproduct_selection = np.zeros(J)
    Xproduct_selected = pd.DataFrame([[0]*T]*J)
    Xshare_product = np.zeros(J)
    Xshare_competition = np.zeros(K)
    Xslack_product = np.zeros(J)
    Xproduction = pd.DataFrame([[0]*T]*J)

    #product selection computation (substitui 3.3)
    Xproduct_selection = (new_solution.sum(axis = 'columns') > 0) * 1

    #binary setup famility computation
    for m in range(0,len(data[15])):
        Xproduct_selected_family.iloc[m,:] = new_solution.iloc[data[15][m]].sum()
        Xproduct_selected_family = (Xproduct_selected_family > 0) * 1

    neighbor_results = quantities_computation(new_solution, Xproduct_selected_family, Xproduct_selection, Xamounts_sold, Xproduction, Xholding_inventory, Xshare_product, Xshare_competition, Xslack_product)
    Xamounts_sold = neighbor_results[0]
    Xproduction = neighbor_results[1]
    Xholding_inventory = neighbor_results[2]
    Xshare_product  = neighbor_results[3]
    Xshare_competition = neighbor_results[4]
    Xslack_product = neighbor_results[5]

    #EVALUATE IF BETTER
    feasible_solution = verify_feasibility(Xamounts_sold, Xshare_product, Xproduct_selection, Xshare_competition, new_solution, Xproduction, Xproduct_selected_family, Xslack_product)
    new_evaluation = evaluate_function(Xamounts_sold, Xholding_inventory, Xproduct_selected, Xproduct_selected_family, Xproduct_selection)
    
    return new_solution, new_evaluation, feasible_solution[0]

# Acceptance probability for simulated annealing
def acceptance_probability(current_evaluation, next_evaluation, temperature):
    if next_evaluation > current_evaluation:
        # If the new state is better than the current state, always accept it
        return 1.0
    else:
        try:        
            # Otherwise, calculate the acceptance probability
            return math.exp((next_evaluation - current_evaluation) / temperature)
        except:
            return 1.0
        
# Perform a simulated annealing search starting from a given initial solution and parameters with guided neighborhood
def simulated_annealing_guided(initial_solution, initial_evaluation, temperature, cooling_rate, cooling_frequency, temperature_limit):
    
    #Initialize dataframe to save results
    results = []
    best = []
    start=datetime.now()
    time = []
    new_evaluation = 0
    
    # Initialize the best solution found so far to the initial solution
    best_solution = initial_solution
    best_evaluation = initial_evaluation
    
    # Initialize a flag to indicate if the search is complete
    iteration_product = 0
    l = 0
    N_iteration_product = 100 # maximum

    # While the search is not complete...
    while temperature > temperature_limit:
        j = random.randint(0,J-1)
        iteration_product = iteration_product + 1
        
        # Generate the product neighborhood of the current solution        
        neighbors = generate_neighborhood_guided(best_solution, j)
        
        # Set the current solution to the best solution in the neighborhood
        current_solution = best_neighbor(neighbors)
        
        # Calculate the acceptance probability
        acceptance = acceptance_probability(best_evaluation, current_solution[1], temperature)
        
        print(str(current_solution[1])+' / '+str(best_evaluation))
        results.append(current_solution[1])
        best.append(best_evaluation)
        time.append(datetime.now()-start)
        
        # Decide whether to move to the next state
        if random.random() < acceptance:
            best_solution = current_solution[0]
            best_evaluation = current_solution[1]
        
        l = l + 1
        if l % cooling_frequency:
            # Cool down the temperature
            temperature *= 1 - cooling_rate

    # Return the best solution found
    return best_solution, results, best, time

# Perform a simulated annealing search starting from a given initial solution and parameters with one flip move
def simulated_annealing_blind(initial_solution, initial_evaluation, temperature, cooling_rate, cooling_frequency, temperature_limit):
    
    #Initialize dataframe to save results
    results = []
    best = []
    feasible = []
    start=datetime.now()
    time = []
    new_evaluation = 0
    
    # Initialize the best solution found so far to the initial solution
    best_solution = initial_solution
    best_evaluation = initial_evaluation
    last_evaluation = 0
    
    # Initialize a flag to indicate if the search is complete
    finish = False
    iteration_product = 0
    l = 0

    # While the search is not complete...
    while temperature > temperature_limit:
        iteration_product = iteration_product + 1
        
        # Set the current solution to the best solution in the neighborhood
        current_solution = generate_neighbor_simulated_annealing(best_solution)
        
        # Calculate the acceptance probability
        acceptance = acceptance_probability(best_evaluation, current_solution[1], temperature)
        random_number = random.randint(0,1000000000000000000) / 1000000000000000000
        
        print('new: ' + str(current_solution[1])) #+' / best: '+str(best_evaluation)+' / feasible: '+str(current_solution[2])+' / temperature: ' + str(temperature) + ' / acceptance: ' + str(acceptance) + ' / random: ' + str(random_number))
        results.append(current_solution[1])
        best.append(best_evaluation)
        time.append(datetime.now()-start)
        feasible.append(current_solution[2])

        # Decide whether to move to the next state
        if (acceptance > random_number) & current_solution[2]:
            last_evaluation = best_evaluation
            best_solution = current_solution[0]
            best_evaluation = current_solution[1]
        
        l = l + 1
        if l % cooling_frequency == 0:
            # Cool down the temperature
            temperature *= 1 - cooling_rate
            '''
            j = random.randint(0,J-1)
            for t in range(0,T):
                best_solution.iloc[j,t] = 0
            print('mudei: ' + str(j))
            '''

    # Return the best solution found
    return best_solution, results, best, time

# Simulated annealing with one flip move and initial peturbation of best solution
def iterated_simulated_annealing_blind(initial_solution, initial_evaluation, limit_tries, Xtemperature, Xcooling_rate, Xcooling_frequency, Xtemperature_limit):
    start=datetime.now()
    iterated_time = []
    iterated_results = []
    iterated_best = []
    
    # Initialize the best solution found so far to the initial solution
    best_solution = initial_solution
    best_evaluation = initial_evaluation
    
    tries = 0
    
    while tries < limit_tries:
        tries = tries + 1
        
        '''
        new_solution = best_solution.copy()
        
        for t in range(0,T):
            for j in range(0,J):
                ku = random.randint(0,10)
                if ku == 1:
                    new_solution.iloc[j, t] = abs(new_solution.iloc[j, t] - 1)
                    
        Xamounts_sold = pd.DataFrame([[0]*T]*J)
        Xholding_inventory = pd.DataFrame([[0]*T]*J)
        Xproduct_selected_family = pd.DataFrame([[0]*T]*M)
        Xproduct_selection = np.zeros(J)
        Xproduct_selected = pd.DataFrame([[0]*T]*J)
        Xshare_product = np.zeros(J)
        Xshare_competition = np.zeros(K)
        Xslack_product = np.zeros(J)
        Xproduction = pd.DataFrame([[0]*T]*J)

        #product selection computation (substitui 3.3)
        Xproduct_selection = (new_solution.sum(axis = 'columns') > 0) * 1

        #binary setup famility computation
        for m in range(0,len(data[15])):
            Xproduct_selected_family.iloc[m,:] = new_solution.iloc[data[15][m]].sum()
            Xproduct_selected_family = (Xproduct_selected_family > 0) * 1
            
        current = quantities_computation(Xproduct_selected, Xproduct_selected_family, Xproduct_selection, Xamounts_sold, Xproduction, Xholding_inventory, Xshare_product, Xshare_competition, Xslack_product)
        Xamounts_sold = current[0]
        Xproduction = current[1]
        Xholding_inventory = current[2]
        Xshare_product  = current[3]
        Xshare_competition = current[4]
        Xslack_product = current[5]

        #EVALUATE IF BETTER
        feasible_solution = verify_feasibility(Xamounts_sold, Xshare_product, Xproduct_selection, Xshare_competition, Xproduct_selected, Xproduction, Xproduct_selected_family, Xslack_product)
        current_evaluation = evaluate_function(Xamounts_sold, Xholding_inventory, Xproduct_selected, Xproduct_selected_family, Xproduct_selection)
        '''
        
        local = simulated_annealing_blind(best_solution, best_evaluation, Xtemperature, Xcooling_rate, Xcooling_frequency, Xtemperature_limit)
        
        print('iterated: '+ str(local[2][-1]) +' / '+ str(best_evaluation))
        iterated_results.append(int(local[2][-1]))
        iterated_best.append(best_evaluation)
        iterated_time.append(datetime.now()-start)
        
        # If the current solution is better than the best solution found so far...
        if local[2][-1] > best_evaluation:
            # Update the best solution found so far
            best_solution = local[0]
            best_evaluation = local[2][-1]

    return best_solution, iterated_results, iterated_best, iterated_time

# Local search with one flip move and initial peturbation of best solution
def iterated_local_search_blind(initial_solution, initial_evaluation, limit_tries):
    start=datetime.now()
    iterated_results = []
    iterated_best = []
    iterated_time =[]
    
    # Initialize the best solution found so far to the initial solution
    best_solution = initial_solution
    best_evaluation = initial_evaluation
    
    tries = 0
    
    while tries < limit_tries:
        tries = tries + 1
        
        new_solution = best_solution.copy()
        for t in range(0,T):
            for j in range(0,J):
                ku = random.randint(0,10)
                if ku == 1:
                    new_solution.iloc[j, t] = abs(new_solution.iloc[j, t] - 1)
                    
        Xamounts_sold = pd.DataFrame([[0]*T]*J)
        Xholding_inventory = pd.DataFrame([[0]*T]*J)
        Xproduct_selected_family = pd.DataFrame([[0]*T]*M)
        Xproduct_selection = np.zeros(J)
        Xproduct_selected = pd.DataFrame([[0]*T]*J)
        Xshare_product = np.zeros(J)
        Xshare_competition = np.zeros(K)
        Xslack_product = np.zeros(J)
        Xproduction = pd.DataFrame([[0]*T]*J)

        #product selection computation (substitui 3.3)
        Xproduct_selection = (new_solution.sum(axis = 'columns') > 0) * 1

        #binary setup famility computation
        for m in range(0,len(data[15])):
            Xproduct_selected_family.iloc[m,:] = new_solution.iloc[data[15][m]].sum()
            Xproduct_selected_family = (Xproduct_selected_family > 0) * 1
            
        current = quantities_computation(Xproduct_selected, Xproduct_selected_family, Xproduct_selection, Xamounts_sold, Xproduction, Xholding_inventory, Xshare_product, Xshare_competition, Xslack_product)
        Xamounts_sold = current[0]
        Xproduction = current[1]
        Xholding_inventory = current[2]
        Xshare_product  = current[3]
        Xshare_competition = current[4]
        Xslack_product = current[5]

        #EVALUATE IF BETTER
        feasible_solution = verify_feasibility(Xamounts_sold, Xshare_product, Xproduct_selection, Xshare_competition, Xproduct_selected, Xproduction, Xproduct_selected_family, Xslack_product)
        current_evaluation = evaluate_function(Xamounts_sold, Xholding_inventory, Xproduct_selected, Xproduct_selected_family, Xproduct_selection)
        
        local = local_search_blind(new_solution, current_evaluation)
        
        print('iterated: '+ str(local[2][-1]) +' / '+ str(best_evaluation))
        iterated_results.append(int(local[2][-1]))
        iterated_best.append(best_evaluation)
        iterated_time.append(datetime.now()-start)
        
        # If the current solution is better than the best solution found so far...
        if local[2][-1] > best_evaluation:
            # Update the best solution found so far
            best_solution = local[0]
            best_evaluation = local[2][-1]

    return best_solution, iterated_results, iterated_best, iterated_time

## Run code with random constructive and selected method

In [ ]:
#text_files = ["12-10-2.txt", "12-18-0.txt"]
text_files = ['12-6-4.txt', '12-6-6.txt', '12-8-1.txt', '12-8-4.txt', '12-10-1.txt','12-10-3.txt']

for u in range(0, len(text_files)):
    #Read file
    data= read_file_unsolved(text_files[u])
    T = data[0]
    J = data[1]
    K = len(data[16])
    M = data[2]
    category_demand_per_period = pd.DataFrame(data[14]).T * data[9]
    max_production = pd.DataFrame([[999999999]*T]*J)

    #3.1
    amounts_sold = pd.DataFrame([[0]*T]*J)
    holding_inventory = pd.DataFrame([[0]*T]*J)
    product_selected_family = pd.DataFrame([[0]*T]*M)

    #3.2
    product_selection = np.zeros(J)
    product_selected = pd.DataFrame([[0]*T]*J)

    #3.4
    share_product = np.zeros(J)
    share_competition = np.zeros(K)
    slack_product = np.zeros(J)

    #3.9
    production = pd.DataFrame([[0]*T]*J)
    
    product_selection[0] = 1
    product_selected_family[0][0] = 1
    product_selected_family[1][0] = 1
    product_selected_family[2][0] = 1
    product_selected_family[3][0] = 1
    product_selected_family[4][0] = 1
    product_selected_family[5][0] = 1
    product_selected_family[6][0] = 1
    product_selected_family[7][0] = 1
    product_selected_family[8][0] = 1
    product_selected_family[9][0] = 1
    product_selected_family[10][0] = 1
    product_selected_family[11][0] = 1
    product_selected[0][0] = 1
    product_selected[1][0] = 1
    product_selected[2][0] = 1
    product_selected[3][0] = 1
    product_selected[4][0] = 1
    product_selected[5][0] = 1
    product_selected[6][0] = 1
    product_selected[7][0] = 1
    product_selected[8][0] = 1
    product_selected[9][0] = 1
    product_selected[10][0] = 1
    product_selected[11][0] = 1
    
    m= gp.Model('PLS')
    #Decision Variables
    p_amounts_sold_t = m.addVars(data[0], data[1], lb=0, vtype=GRB.CONTINUOUS)
    p_production_t = m.addVars(data[0],data[1],lb=0, vtype=GRB.CONTINUOUS)
    p_holding_t = m.addVars(data[0],data[1],lb=0, vtype=GRB.CONTINUOUS)
    p_share = m.addVars(data[1],lb=0, ub=1, vtype=GRB.CONTINUOUS) 
    k_share_competition = m.addVars(data[18],lb=0,ub=1, vtype=GRB.CONTINUOUS)
    p_slack= m.addVars(data[1],lb=0,ub=1000,vtype=GRB.CONTINUOUS)
    #constraints
    c1=c2=c3=c4=c5=c6=c7=c8=c9 = list()

    for i in range(0, data[0]):
        #soft - capacity
        c8.append(m.addConstr(gp.quicksum(data[12][fam]*product_selected_family[i][fam] for fam in range(0,data[2])) + gp.quicksum(data[8][prod]*product_selected[i][prod] + p_production_t[i,prod]*data[6][prod] for prod in range(0,data[1])) <= data[10][i]))
        for j in range(0, data[1]):
            c1.append( m.addConstr(p_amounts_sold_t[i,j] - (p_share[j])* category_demand_per_period[i][p_belongs_k(j)] <= 0 ))
            c7.append( m.addConstr(p_production_t[i,j] - 99999999999999 * product_selected[i][j] <= 0 ))
            #c8.append( m.addConstr(p_holding_t[i,j] - 9999999999999999 * holding_selected[i][j] <= 0 ))
            if i!=0:
                c6.append(m.addConstr(p_amounts_sold_t[i,j] == p_production_t[i,j] + p_holding_t[i-1,j] - p_holding_t[i,j]))
            else:
                c6.append(m.addConstr(p_amounts_sold_t[i,j] == p_production_t[i,j] - p_holding_t[i,j]))

    #3.3 c2
    for j in range(0, data[1]):
        c2.append(m.addConstr(p_share[j] <= product_selection[j]))
        c9.append(m.addConstr(p_share[j] >= product_selection[j]*0.000000000000000001))

    #3.4 c3
    #3.5 c4
    #3.6 c5
    for k in range(0,len(data[16])):
        array = np.array(data[16][k])
        c5.append(m.addConstr(k_share_competition[k] + gp.quicksum(p_share[aux] for aux in data[16][k]) == 1 ))
        for j in range(0,len(array)):
            c3.append(m.addConstr(p_slack[array[j]] == data[17][array[j]] * k_share_competition[k] - p_share[array[j]] * data[11][k]))
            c4.append(m.addConstr(p_slack[array[j]] <= data[17][array[j]] * k_share_competition[k]) )

    #objective function
    R = gp.quicksum(data[3][product] * p_amounts_sold_t[period,product] for period in range(0,data[0]) for product in range(0,data[1]))
    C1 = gp.quicksum(data[4][product] * p_holding_t[period,product] for period in range(0,data[0]) for product in range(0,data[1]))
    C2 = sum(product_selected.sum(axis = 'columns') * data[5])
    C3 = sum(product_selected_family.sum(axis = 'columns') * data[13])
    C4 = sum(product_selection * data[7])
    obj = R - C1 - C2 - C3 - C4
    m.setObjective(obj, GRB.MAXIMIZE)
    # Suppress Gurobi output by setting the LogToConsole parameter to 0
    m.params.LogToConsole = 0
    m.optimize()

    for j_match in range(0,data[1]):
        for t_match in range(0,data[0]):
            amounts_sold[t_match][j_match] = p_amounts_sold_t[t_match,j_match].X
            production[t_match][j_match] = p_production_t[t_match,j_match].X     
            holding_inventory[t_match][j_match] = p_holding_t[t_match,j_match].X     
        share_product[j_match] = p_share[j_match].X
        slack_product[j_match] = p_slack[j_match].X

    for k_match in range(0,K):     
        share_competition[k_match] = k_share_competition[k_match].X

    verify_feasibility(amounts_sold, share_product, product_selection, share_competition, product_selected, production, product_selected_family, slack_product)
    evaluate_function(amounts_sold, holding_inventory, product_selected, product_selected_family, product_selection)
    current_evaluation = obj.getValue()
    obj.getValue()
    
    sa_2 = local_search_blind(product_selected, current_evaluation)
    #local_2[2][-1]
    temperature = 1000
    #cooling_rate = 0.005
    cooling_rate = 0.01 #less than 0.15 is suggested in literature, but 0.01 is common practice
    cooling_rate = 0.05
    cooling_frequency = 5 #literature says between 5 and 10. experiments showed it is not worth it 
    temperature_limit = 1 #0.1 is suggested in literature but 1 is enough
    #sa_2 = simulated_annealing_blind(product_selected, current_evaluation, temperature, cooling_rate, cooling_frequency, temperature_limit)
    #sa_2 = iterated_local_search_blind(product_selected, current_evaluation, 20)
    
    workbook = openpyxl.load_workbook('ResultsIteratedSASearchBlind_1.xlsx')
    worksheet = workbook.create_sheet(text_files[u])
   # Write the matrix to the worksheet
    matrix = sa_2[0]
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            worksheet.cell(row=i+1, column=j+1).value = matrix[i][j]

    # Add a blank row between the matrix and arrays
    worksheet.cell(row=len(matrix)+1, column=1).value = ""

    # Write the title row for the arrays
    worksheet.cell(row=len(matrix)+2, column=1).value = "Best Evaluation"
    worksheet.cell(row=len(matrix)+2, column=2).value = "Current Evaluation"
    worksheet.cell(row=len(matrix)+2, column=3).value = "Run Time"

    # Write the arrays to the worksheet
    array1 = sa_2[1]
    array2 = sa_2[2]
    array3 = sa_2[3]
    for i in range(len(array1)):
        worksheet.cell(row=len(matrix)+i+3, column=1).value = array1[i]
        worksheet.cell(row=len(matrix)+i+3, column=2).value = array2[i]
        worksheet.cell(row=len(matrix)+i+3, column=3).value = array3[i]

    # Save the workbook
    workbook.save('ResultsIteratedSASearchBlind_1.xlsx')  

## Run code with constructive and selected method

In [ ]:
text_files = ['12-8-1.txt']

for u in range(0, len(text_files)):
    #Read file
    data= read_file_unsolved(text_files[u])
    T = data[0]
    J = data[1]
    K = len(data[16])
    M = data[2]
    category_demand_per_period = pd.DataFrame(data[14]).T * data[9]
    max_production = pd.DataFrame([[999999999]*T]*J)

    #3.1
    amounts_sold = pd.DataFrame([[0]*T]*J)
    holding_inventory = pd.DataFrame([[0]*T]*J)
    product_selected_family = pd.DataFrame([[0]*T]*M)

    #3.2
    product_selection = np.zeros(J)
    product_selected = pd.DataFrame([[0]*T]*J)

    #3.4
    share_product = np.zeros(J)
    share_competition = np.zeros(K)
    slack_product = np.zeros(J)

    #3.9
    production = pd.DataFrame([[0]*T]*J)
    
    #Constructive
    
    array_added = []
    family_added = []
    positive_return = True
    max_capacity = T * np.mean(data[10])

    while positive_return:
        biggest_weight = 0
        new_weight = 0
        j_of_biggest = -1

        # get capacity used
        capacity_used_basis = 0 

        product_selection = np.zeros(J)
        product_selection[array_added] = 1

        for k in range(0,len(data[16])):
            array = np.array(data[16][k])
            for j in range(0,len(array)):           
                if [array[j]] in array_added:
                    #get family
                    for m_match in range(0,len(data[15])):
                        array_match = np.array(data[15][m_match])
                        for j_match in range(0,len(array_match)):
                            if array_match[j_match] == array[j]:
                                m = m_match
                                n_products_in_family = len(array_match)
                    #get share
                    share = share_computation(product_selection, array[j])
                    amounts_sold.iloc[array[j],:] = share * category_demand_per_period.loc[k,:]
                    if m not in family_added:
                        capacity_used_basis = capacity_used_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]] + data[12][m])
                    else:
                        capacity_used_basis = capacity_used_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]])

        #for each category
        for k in range(0,len(data[16])):
            array = np.array(data[16][k])

            #calculate basis of products selected
            for j in range(0,len(array)):
                numerator_basis = 0
                denominator_basis = 0
                if [array[j]] in array_added:
                    #get family
                    for m_match in range(0,len(data[15])):
                        array_match = np.array(data[15][m_match])
                        for j_match in range(0,len(array_match)):
                            if array_match[j_match] == array[j]:
                                m = m_match
                                n_products_in_family = len(array_match)

                    #get share
                    share = share_computation(product_selection, array[j])
                    #weight
                    amounts_sold.iloc[array[j],:] = share * category_demand_per_period.loc[k,:]
                    if m not in family_added:
                        numerator_basis = numerator_basis + sum(amounts_sold.iloc[array[j],:]) * data[3][array[j]] - 12 * (data[5][array[j]] + data[13][m]) - data[7][array[j]]
                        denominator_basis = denominator_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]] + data[12][m])
                    else:
                        numerator_basis = numerator_basis + sum(amounts_sold.iloc[array[j],:]) * data[3][array[j]] - 12 * (data[5][array[j]]) - data[7][array[j]]
                        denominator_basis = denominator_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]])

            #compute weight of new product and check if better than last
            for j in range(0,len(array)):
                if [array[j]] not in array_added:
                    product_selection = np.zeros(J)
                    product_selection[array_added] = 1
                    product_selection[array[j]] = 1
                    #get family
                    for m_match in range(0,len(data[15])):
                        array_match = np.array(data[15][m_match])
                        for j_match in range(0,len(array_match)):
                            if array_match[j_match] == array[j]:
                                m = m_match
                                n_products_in_family = len(array_match)
                    #get share
                    share = share_computation(product_selection, array[j])
                    #weight
                    amounts_sold.iloc[array[j],:] = share * category_demand_per_period.loc[k,:]
                    if m not in family_added:
                        numerator = numerator_basis + sum(amounts_sold.iloc[array[j],:]) * data[3][array[j]] - 12 * (data[5][array[j]] + data[13][m]) - data[7][array[j]]
                        denominator = denominator_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]] + data[12][m])
                        capacity_used = capacity_used_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]] + data[12][m])
                    else:
                        numerator = numerator_basis + sum(amounts_sold.iloc[array[j],:]) * data[3][array[j]] - 12 * (data[5][array[j]]) - data[7][array[j]]
                        denominator = denominator_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]])
                        capacity_used = capacity_used_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]])
                    new_weight = numerator / denominator
                else:
                    new_weight = 0

                if (new_weight > biggest_weight) & (capacity_used < max_capacity):
                    biggest_weight = new_weight
                    j_of_biggest = array[j]
                    m_of_biggest = m

        #add best
        if j_of_biggest > -1:            
            array_added.append(j_of_biggest)
            if m_of_biggest not in family_added:
                family_added.append(m_of_biggest)           
        else:
            positive_return = False

    product_selection = np.zeros(J)
    product_selection[array_added] = 1

    share_model = gp.Model('Share_optmization')

    #decisionvariables
    p_share = share_model.addVars(data[1],lb=0, ub=1, vtype=GRB.CONTINUOUS) 
    k_share_competition = share_model.addVars(data[18],lb=0,ub=1, vtype=GRB.CONTINUOUS)
    p_slack= share_model.addVars(data[1],lb=0,ub=1000000,vtype=GRB.CONTINUOUS)

    #constraints
    c1=c2=c3=c4=c5 = list()

    for i in range(0,data[1]):
        c1 = share_model.addConstr(p_share[i]<=product_selection[i])
        c5 = share_model.addConstr(p_share[i]>=product_selection[i]*0.000000000000001)

    for k in range(0,len(data[16])):
        array = np.array(data[16][k])
        c4.append(share_model.addConstr(k_share_competition[k] + gp.quicksum(p_share[aux] for aux in data[16][k]) == 1 ))
        for j in range(0,len(array)):
            c2.append(share_model.addConstr(p_slack[array[j]] == data[17][array[j]] * k_share_competition[k] - p_share[array[j]] * data[11][k]))
            c3.append(share_model.addConstr(p_slack[array[j]] <= data[17][array[j]] * k_share_competition[k]) )

    #objective function
    sum_shares = gp.quicksum(p_share[prod] for prod in range(0,data[1])) - gp.quicksum(k_share_competition[comp] for comp in range(0,data[18]))
    share_model.setObjective(sum_shares, GRB.MAXIMIZE)
    # Suppress Gurobi output by setting the LogToConsole parameter to 0
    share_model.params.LogToConsole = 0
    share_model.optimize()

    for j in range(0,J):
        slack_product[j] = p_slack[j].X
        share_product[j] = p_share[j].X

    for k in range(0,K):
        share_competition[k] = k_share_competition[k].X

    #amounts sold computation (substituiu 3.2)
    for k in range(0,len(data[16])):
        array = np.array(data[16][k])
        for j in range(0,len(array)):
            amounts_sold.iloc[array[j],:] = share_product[array[j]] * category_demand_per_period.loc[k,:]

    original_amounts_sold = amounts_sold.copy()

    for t in range(T-1,0,-1):
        holding_array = []
        for j in array_added:
            holding_array.append(data[4][j])
        production_planning = amounts_sold.iloc[array_added,:]
        for j in range(0,len(array_added)):
            production_planning.iloc[j,:] = production_planning.iloc[j,:] * holding_array[j]
        sorted_ = production_planning[t].sort_values(ascending = False)
        array_not_added_production = []
        capacity_used = 0
        current_family_added = []
        capacity_used = 0
        for j in range(0,len(sorted_)): 
            temp = 0
            for m_match in range(0,len(data[15])):
                    array_match = np.array(data[15][m_match])
                    for j_match in range(0,len(array_match)):
                        if array_match[j_match] == np.array(sorted_.index)[j]:
                            m = m_match
            if (m not in current_family_added) & (m in family_added):
                current_family_added.append(m)
                temp = data[12][m]    
            temp = temp + amounts_sold.iloc[np.array(sorted_.index)[j],t] * data[8][np.array(sorted_.index)[j]] + (data[6][np.array(sorted_.index)[j]])
            if (capacity_used + temp) > data[10][t]:
                array_not_added_production.append(np.array(sorted_.index)[j])
            else:
                capacity_used = capacity_used + temp

        final_to_add = -9999999999999999999999
        amount_holding = 0
        weight_selected = -9999999999999999999999

        for j in array_not_added_production:
            capacity_remaining = data[10][t] - capacity_used
            for m_match in range(0,len(data[15])):
                    array_match = np.array(data[15][m_match])
                    for j_match in range(0,len(array_match)):
                        if array_match[j_match] == j:
                            m = m_match
            if (m not in current_family_added) & (m in family_added):
                amount_that_fits = (capacity_remaining - (data[6][j] + data[12][m])) / data[8][j]
            else:
                amount_that_fits = (capacity_remaining - (data[6][j]) ) / data[8][j]

            if amount_that_fits > 0:
                amount_that_fits = 0

            amount_excess = (amounts_sold.iloc[j,t] - amount_that_fits)
            weight = amount_excess * data[4][j] - data[5][j]
            #holding cost * excess amount - setup cost
            if (weight > weight_selected) & (amount_that_fits > 0):
                final_to_add = j
                amount_holding = amount_excess

        for j in array_not_added_production:
            if j == final_to_add:
                amounts_sold.iloc[j,t] = amounts_sold.iloc[j,t] - amount_holding
                amounts_sold.iloc[j,t-1] = amounts_sold.iloc[j,t-1] + amount_holding
            else:
                amounts_sold.iloc[j,t-1] = amounts_sold.iloc[j,t-1] + amounts_sold.iloc[j,t]
                amounts_sold.iloc[j,t] = 0

    production = amounts_sold.copy()
    amounts_sold = original_amounts_sold.copy()

    #product selected period computation
    product_selected = (production > 0) * 1

    #binary setup famility computation
    for m in range(0,len(data[15])):
        product_selected_family.iloc[m,:] = product_selected.iloc[data[15][m]].sum()
        product_selected_family = (product_selected_family > 0) * 1

    #holding inventory computation
    holding_inventory.iloc[:,T-1] = 0
    for t in range(T-2,-1,-1):
        holding_inventory.iloc[:,t] = amounts_sold.iloc[:,t+1] - production.iloc[:,t+1] + holding_inventory.iloc[:,t+1]

    for j in range(0,holding_inventory.shape[0]):
        for t in range(0,holding_inventory.shape[1]):
            if holding_inventory.iloc[j,t] < 0.01:
                holding_inventory.iloc[j,t] = 0

    for t in range(T-1,0,-1):
        capacity_used = 0
        for j in array_added:
            capacity_used = capacity_used + production.iloc[j,t] * data[8][j] + (data[6][j]) * product_selected.iloc[j,t]

        for m in family_added:
            capacity_used = capacity_used + data[12][m] * product_selected_family.iloc[m,t]

        if capacity_used > data[10][t]:
            diff = capacity_used - data[10][t]
            print(t)
            print(diff)

    BESTamounts_sold = amounts_sold
    BESTholding_inventory = holding_inventory
    BESTproduct_selection = product_selection
    BESTproduct_selected_family = product_selected_family
    BESTproduct_selected = product_selected
    BESTshare_product = share_product
    BESTshare_competition = share_competition
    BESTslack_product = slack_product
    BESTproduction = production
    feasible_solution = verify_feasibility(BESTamounts_sold, BESTshare_product, BESTproduct_selection, BESTshare_competition, BESTproduct_selected, BESTproduction, BESTproduct_selected_family, BESTslack_product)
    current_evaluation = evaluate_function(BESTamounts_sold, BESTholding_inventory, BESTproduct_selected, BESTproduct_selected_family, BESTproduct_selection)
    
    m= gp.Model('PLS')
    #Decision Variables
    p_amounts_sold_t = m.addVars(data[0], data[1], lb=0, vtype=GRB.CONTINUOUS)
    p_production_t = m.addVars(data[0],data[1],lb=0, vtype=GRB.CONTINUOUS)
    p_holding_t = m.addVars(data[0],data[1],lb=0, vtype=GRB.CONTINUOUS)
    p_share = m.addVars(data[1],lb=0, ub=1, vtype=GRB.CONTINUOUS) 
    k_share_competition = m.addVars(data[18],lb=0,ub=1, vtype=GRB.CONTINUOUS)
    p_slack= m.addVars(data[1],lb=0,ub=1000,vtype=GRB.CONTINUOUS)
    #constraints
    c1=c2=c3=c4=c5=c6=c7=c8=c9 = list()

    for i in range(0, data[0]):
        #soft - capacity
        c8.append(m.addConstr(gp.quicksum(data[12][fam]*product_selected_family[i][fam] for fam in range(0,data[2])) + gp.quicksum(data[8][prod]*product_selected[i][prod] + p_production_t[i,prod]*data[6][prod] for prod in range(0,data[1])) <= data[10][i]))
        for j in range(0, data[1]):
            c1.append( m.addConstr(p_amounts_sold_t[i,j] - (p_share[j])* category_demand_per_period[i][p_belongs_k(j)] <= 0 ))
            c7.append( m.addConstr(p_production_t[i,j] - 99999999999999 * product_selected[i][j] <= 0 ))
            #c8.append( m.addConstr(p_holding_t[i,j] - 9999999999999999 * holding_selected[i][j] <= 0 ))
            if i!=0:
                c6.append(m.addConstr(p_amounts_sold_t[i,j] == p_production_t[i,j] + p_holding_t[i-1,j] - p_holding_t[i,j]))
            else:
                c6.append(m.addConstr(p_amounts_sold_t[i,j] == p_production_t[i,j] - p_holding_t[i,j]))

    #3.3 c2
    for j in range(0, data[1]):
        c2.append(m.addConstr(p_share[j] <= product_selection[j]))
        c9.append(m.addConstr(p_share[j] >= product_selection[j]*0.000000000000000001))

    #3.4 c3
    #3.5 c4
    #3.6 c5
    for k in range(0,len(data[16])):
        array = np.array(data[16][k])
        c5.append(m.addConstr(k_share_competition[k] + gp.quicksum(p_share[aux] for aux in data[16][k]) == 1 ))
        for j in range(0,len(array)):
            c3.append(m.addConstr(p_slack[array[j]] == data[17][array[j]] * k_share_competition[k] - p_share[array[j]] * data[11][k]))
            c4.append(m.addConstr(p_slack[array[j]] <= data[17][array[j]] * k_share_competition[k]) )

    #objective function
    R = gp.quicksum(data[3][product] * p_amounts_sold_t[period,product] for period in range(0,data[0]) for product in range(0,data[1]))
    C1 = gp.quicksum(data[4][product] * p_holding_t[period,product] for period in range(0,data[0]) for product in range(0,data[1]))
    C2 = sum(product_selected.sum(axis = 'columns') * data[5])
    C3 = sum(product_selected_family.sum(axis = 'columns') * data[13])
    C4 = sum(product_selection * data[7])
    obj = R - C1 - C2 - C3 - C4
    m.setObjective(obj, GRB.MAXIMIZE)
    # Suppress Gurobi output by setting the LogToConsole parameter to 0
    m.params.LogToConsole = 0
    m.optimize()

    for j_match in range(0,data[1]):
        for t_match in range(0,data[0]):
            amounts_sold[t_match][j_match] = p_amounts_sold_t[t_match,j_match].X
            production[t_match][j_match] = p_production_t[t_match,j_match].X     
            holding_inventory[t_match][j_match] = p_holding_t[t_match,j_match].X     
        share_product[j_match] = p_share[j_match].X
        slack_product[j_match] = p_slack[j_match].X

    for k_match in range(0,K):     
        share_competition[k_match] = k_share_competition[k_match].X

    verify_feasibility(amounts_sold, share_product, product_selection, share_competition, product_selected, production, product_selected_family, slack_product)
    evaluate_function(amounts_sold, holding_inventory, product_selected, product_selected_family, product_selection)
    current_evaluation = obj.getValue()
    obj.getValue()
    
    #sa_2 = local_search_blind(product_selected, current_evaluation)
    #local_2[2][-1]
    temperature = 50
    #cooling_rate = 0.005
    cooling_rate = 0.005 #less than 0.15 is suggested in literature, but 0.01 is common practice
    #cooling_rate = 0.05
    cooling_frequency = 10 #literature says between 5 and 10. experiments showed it is not worth it 
    temperature_limit = 1 #0.1 is suggested in literature but 1 is enough
    #sa_2 = iterated_simulated_annealing_blind(product_selected, current_evaluation, 100, temperature, cooling_rate, cooling_frequency, temperature_limit)
    sa_2 = simulated_annealing_blind(product_selected, current_evaluation, temperature, cooling_rate, cooling_frequency, temperature_limit)
    #sa_2 = iterated_local_search_blind(product_selected, current_evaluation, 30)
    
    workbook = openpyxl.load_workbook('ResultsIteratedSASearchBlind_1.xlsx')
    worksheet = workbook.create_sheet(text_files[u])
   # Write the matrix to the worksheet
    matrix = sa_2[0]
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            worksheet.cell(row=i+1, column=j+1).value = matrix[i][j]

    # Add a blank row between the matrix and arrays
    worksheet.cell(row=len(matrix)+1, column=1).value = ""

    # Write the title row for the arrays
    worksheet.cell(row=len(matrix)+2, column=1).value = "Best Evaluation"
    worksheet.cell(row=len(matrix)+2, column=2).value = "Current Evaluation"
    worksheet.cell(row=len(matrix)+2, column=3).value = "Run Time"

    # Write the arrays to the worksheet
    array1 = sa_2[1]
    array2 = sa_2[2]
    array3 = sa_2[3]
    for i in range(len(array1)):
        worksheet.cell(row=len(matrix)+i+3, column=1).value = array1[i]
        worksheet.cell(row=len(matrix)+i+3, column=2).value = array2[i]
        worksheet.cell(row=len(matrix)+i+3, column=3).value = array3[i]

    # Save the workbook
    workbook.save('ResultsIteratedSASearchBlind_1.xlsx')

new: 6712.121424023098
new: 6161.263664968659
new: 6712.121424023098
new: 6766.5886844243105
new: 6380.176254901438
new: 6766.5886844243105
new: 6673.965068032957
new: 6528.507607821434
new: 6572.853124978963
new: 6522.101991810272
new: 6300.80209404597
new: 6598.946043929297
new: 6567.59762132277
new: 6275.094293769191
new: 6567.59762132277
new: 6497.118466910161
new: 6567.59762132277
new: 6117.479468080046
new: 6315.540346424919
new: 6343.5378970654865
new: 6296.8044954432125
new: 6473.41744784572
new: 6567.59762132277
new: 6496.282235594869
new: 6434.356760399365
new: 6512.55099332546
new: 6481.684814382739
new: 6480.464031121236
new: 6511.984105222929
new: 6622.345271076914
new: 6577.700246829154
new: 6366.711629637081
new: 6410.09628433724
new: 6266.812765750953
new: 6508.927468816663
new: 6628.396500769994
new: 6387.053385505647
new: 6524.875936299823
new: 6173.181766923929
new: 6538.902394360681
new: 6591.107199969762
new: 6103.779954418774
new: 6628.396500769994
new: 6633.21283

new: 6119.072729334216
new: 6574.908517397058
new: 6523.359166255708
new: 6576.571375502119
new: 6576.355045881895
new: 6490.764017986028
new: 5922.870401734971
new: 6576.355045881895
new: 6382.510064693262
new: 6582.697014017576
new: 6064.799355781803
new: 6600.107146440257
new: 6442.68312612599
new: 6441.866419593131
new: 6446.124869412161
new: 6348.035539491528
new: 6536.910076936096
new: 6439.138488096152
new: 6542.157229505078
new: 6551.349069951298
new: 6434.058587901839
new: 6606.962586051138
new: 6367.397728582497
new: 6588.974999264121
new: 6483.321401741466
new: 6565.7172912589285
new: 6567.128938580981
new: 6567.128938580981
new: 6567.128938580981
new: 6514.0116130991255
new: 6567.128938580981
new: 6378.9433044452
new: 6250.0734612163005
new: 6567.128938580981
new: 6640.757098080076
new: 6558.828359464275
new: 6640.757098080076
new: 6643.703738266953
new: 6593.745635410273
new: 6350.8738571572385
new: 6588.090222167114
new: 6479.760700145941
new: 6643.703738266953
new: 6097.

new: 6609.959486438808
new: 6589.424955569185
new: 6423.870027509476
new: 6689.9314528406085
new: 6646.47244058393
new: 6689.9314528406085
new: 6549.968719298218
new: 6610.380520769236
new: 6704.981724623832
new: 6704.981724623832
new: 6745.443891368651
new: 6512.091356624344
new: 6691.9766051056795
new: 6745.443891368651
new: 6919.133766644951
new: 6692.3899799982955
new: 6702.635372984575
new: 6911.903205215342
new: 6870.65733371575
new: 6919.133766644951
new: 6851.1044310941215
new: 6614.035286418506
new: 6919.133766644951
new: 6835.950341114167
new: 6758.880982450027
new: 6591.002901983768
new: 6835.950341114167
new: 6835.950341114167
new: 6791.945044404918
new: 6786.267466920117
new: 6706.31666454013
new: 6653.109666192455
new: 6780.061074522968
new: 6611.659905840406
new: 6697.165559148348
new: 6393.851630034112
new: 6389.464586644562
new: 6691.289902196247
new: 6454.9817193265835
new: 6673.45081650627
new: 6691.289902196247
new: 6605.9244435473
new: 6796.54326426469
new: 6796.54

new: 6792.139432168765
new: 6464.0034688829
new: 6837.240000141595
new: 7030.790397487549
new: 6818.523499051288
new: 6960.928781899869
new: 6821.974131961095
new: 6785.132875243012
new: 6864.07132468453
new: 6948.325606614615
new: 6811.92057552372
new: 6850.11449175521
new: 6791.297247045893
new: 6948.325606614615
new: 6936.946049849157
new: 6935.138447892388
new: 6597.7431637798945
new: 6843.3908941507925
new: 6884.800803008184
new: 6837.654862363468
new: 6868.9671147121035
new: 6874.146425470155
new: 6885.316784641756
new: 6935.138447892388
new: 6935.138447892388
new: 6935.138447892388
new: 6787.408022280232
new: 6935.138447892388
new: 6638.466068031951
new: 6415.140517107262
new: 6838.168903618769
new: 6935.138447892388
new: 6894.853304505088
new: 6861.269030689269
new: 6882.232242484981
new: 6894.853304505088
new: 6932.776056832912
new: 6927.487329712612
new: 6927.487329712612
new: 6893.583129394334
new: 6728.143487691434
new: 6751.23044861856
new: 6796.876690769894
new: 6893.5831

new: 6994.467955921462
new: 6727.920208097296
new: 6757.361781438688
new: 6994.467955921462
new: 6633.959821004566
new: 6617.924508061795
new: 6927.900169761678
new: 6912.3176237552325
new: 6934.793902003685
new: 6934.793902003685
new: 6941.357128884545
new: 6878.033753282609
new: 6934.793902003685
new: 6593.505823470552
new: 6703.596455659124
new: 6810.431623471643
new: 6934.793902003685
new: 6934.793902003685
new: 6603.408573382094
new: 6934.793902003685
new: 6812.80347093326
new: 6850.125294100651
new: 6484.74066841334
new: 6846.177087659197
new: 6934.793902003685
new: 6856.537233447752
new: 6934.793902003685
new: 6846.699333847273
new: 6934.793902003685
new: 6885.897888614334
new: 6934.793902003685
new: 6599.197990621768
new: 6610.574287099018
new: 6584.649436065756
new: 6957.881612834659
new: 6812.723638403069
new: 6924.124694460984
new: 6890.365004436763
new: 6957.881612834659
new: 6873.467551224463
new: 6868.5728194195535
new: 6689.805440462756
new: 7027.113973628577
new: 6860.6

new: 6639.89480913843
new: 6477.404401665826
new: 6814.618718595075
new: 6632.86429094829
new: 6500.79447177185
new: 6836.209527320288
new: 6409.764707074575
new: 6854.508667461046
new: 6959.35127954241
new: 6683.6168021969015
new: 6787.026607081615
new: 6964.78322984503
new: 6975.230820021161
new: 6975.230820021161
new: 6642.938072867593
new: 6937.515572259101
new: 6701.887948455329
new: 7013.63325770656
new: 7013.63325770656
new: 6866.136289680902
new: 7013.63325770656
new: 6994.250963879121
new: 6986.132131694053
new: 6922.269706575831
new: 6890.7787909217805
new: 6821.052341417851
new: 7013.63325770656
new: 6887.638975236407
new: 7013.63325770656
new: 6731.340452999448
new: 6881.53284551458
new: 6942.373979578637
new: 7010.2014476677605
new: 6832.131718630582
new: 7007.709967060854
new: 6952.464504225242
new: 6912.619900846819
new: 7007.709967060854
new: 6719.00063904139
new: 6615.411570028973
new: 6917.879609332915
new: 7007.709967060854
new: 6799.9373302296535
new: 6802.145866616

new: 6998.569060488462
new: 7220.441844572007
new: 7125.293023457979
new: 7220.441844572007
new: 6453.80346020692
new: 6827.796383942569
new: 7220.441844572007
new: 7220.441844572007
new: 6801.298227983918
new: 6992.551549726501
new: 7217.6973452007705
new: 7217.6973452007705
new: 7082.602196245456
new: 6846.042914045888
new: 7217.6973452007705
new: 6937.665837227801
new: 7217.6973452007705
new: 6976.026102127159
new: 7164.973966945839
new: 6916.460012890209
new: 6970.28901218252
new: 7178.792639511833
new: 6773.552959529953
new: 7159.197385995291
new: 6744.137768342087
new: 7140.580976087112
new: 6779.0516029379205
new: 7195.120068134932
new: 7055.095829100928
new: 6962.951054825532
new: 6947.569691801861
new: 7047.276725139472
new: 7217.6973452007705
new: 7180.872293380654
new: 6883.765248339572
new: 6977.245319089125
new: 7168.214019002818
new: 7147.883208046982
new: 6826.820148839875
new: 7088.537208514409
new: 6914.947874890885
new: 7131.246773203008
new: 6908.057837791701
new: 71

new: 7236.40469880462
new: 7111.716436667154
new: 6979.992277850252
new: 7291.214156675836
new: 7375.54270436077
new: 7230.608258602446
new: 7358.499428564861
new: 7230.062730668941
new: 7375.54270436077
new: 7181.707601910767
new: 6656.229434844405
new: 7070.245789907524
new: 7375.54270436077
new: 6499.718382231297
new: 7028.4267661424365
new: 7284.875954198294
new: 7100.216077351101
new: 7228.646098842306
new: 6807.681799581942
new: 7115.306628797508
new: 7018.74086219978
new: 7233.929644749888
new: 7178.908849949512
new: 6699.234178838646
new: 7375.54270436077
new: 7150.406030462337
new: 7375.54270436077
new: 6980.827310157016
new: 7375.54270436077
new: 7375.54270436077
new: 7286.7217072069125
new: 7154.662870490287
new: 6972.686163193948
new: 7223.722137603174
new: 7375.54270436077
new: 7203.70404554389
new: 7375.54270436077
new: 7187.933766647908
new: 7129.936878180191
new: 7034.0073996144465
new: 6791.647544373641
new: 7044.963139700355
new: 7309.694586039151
new: 7375.5427043607

new: 7260.299728603526
new: 7417.467686441677
new: 7491.545005608479
new: 7491.545005608479
new: 7491.545005608479
new: 7491.545005608479
new: 7294.208636988705
new: 7419.319812973185
new: 7414.21992009002
new: 7314.51129620695
new: 7491.545005608479
new: 6952.832752504995
new: 7491.545005608479
new: 7491.545005608479
new: 7228.558188397775
new: 7413.601621041336
new: 7272.01022623016
new: 7491.545005608479
new: 7322.470841375533
new: 7264.4873729860465
new: 7059.601316498841
new: 7000.889995821367
new: 7491.545005608479
new: 7408.053132128611
new: 7198.100094650161
new: 7139.297464957678
new: 7478.5167089985225
new: 6937.568892641389
new: 7272.114974800576
new: 7317.340009592445
new: 7262.216764107786
new: 7471.3097003093535
new: 7491.545005608479
new: 7491.545005608479
new: 7338.987119417459
new: 7491.545005608479
new: 7144.929012939788
new: 7266.143600509842
new: 7328.040880597427
new: 7314.94999732888
new: 7491.545005608479
new: 7192.266802881117
new: 7491.545005608479
new: 7409.69

new: 6976.802567734872
new: 7417.584445661331
new: 7263.63480295558
new: 7522.789180698735
new: 7509.361203534652
new: 6795.128336701562
new: 7588.738326739487
new: 7276.907786293458
new: 7152.0589402918185
new: 7588.738326739487
new: 7290.9008359835225
new: 7364.206354087568
new: 7257.242023936154
new: 7450.052833627797
new: 7588.738326739487
new: 7588.738326739487
new: 7042.00770608808
new: 7339.23554124403
new: 7165.542906905239
new: 7241.147176504864
new: 7173.027784169184
new: 7550.765640054571
new: 7451.438620225972
new: 7267.832649588927
new: 7355.376642736961
new: 6929.6907498826295
new: 7323.416177407448
new: 7367.46396620553
new: 7265.075507595105
new: 7487.711280252209
new: 7018.524052290041
new: 7450.357916633712
new: 7588.738326739487
new: 7387.654590912132
new: 7588.738326739487
new: 7588.738326739487
new: 7536.715850764618
new: 7501.273851832728
new: 7536.01494848456
new: 6711.263140185767
new: 7460.633296803144
new: 7383.993663100594
new: 7353.890997476121
new: 7153.272

## Upload a specific instance

In [ ]:
data = read_file_unsolved('12-6-0.txt')
#data = read_file('12-21-0.txt')

T = data[0]
J = data[1]
K = len(data[16])
M = data[2]
category_demand_per_period = pd.DataFrame(data[14]).T * data[9]
max_production = pd.DataFrame([[999999999]*T]*J)

#3.1
amounts_sold = pd.DataFrame([[0]*T]*J)
holding_inventory = pd.DataFrame([[0]*T]*J)
product_selected_family = pd.DataFrame([[0]*T]*M)

#3.2
product_selection = np.zeros(J)
product_selected = pd.DataFrame([[0]*T]*J)

#3.4
share_product = np.zeros(J)
share_competition = np.zeros(K)
slack_product = np.zeros(J)

#3.9
production = pd.DataFrame([[0]*T]*J)

## Constructive

### Random

In [ ]:
product_selection[0] = 1
product_selected_family[0][0] = 1
product_selected_family[1][0] = 1
product_selected_family[2][0] = 1
product_selected_family[3][0] = 1
product_selected_family[4][0] = 1
product_selected_family[5][0] = 1
product_selected_family[6][0] = 1
product_selected_family[7][0] = 1
product_selected_family[8][0] = 1
product_selected_family[9][0] = 1
product_selected_family[10][0] = 1
product_selected_family[11][0] = 1
product_selected[0][0] = 1
product_selected[1][0] = 1
product_selected[2][0] = 1
product_selected[3][0] = 1
product_selected[4][0] = 1
product_selected[5][0] = 1
product_selected[6][0] = 1
product_selected[7][0] = 1
product_selected[8][0] = 1
product_selected[9][0] = 1
product_selected[10][0] = 1
product_selected[11][0] = 1

m= gp.Model('PLS')
#Decision Variables
p_amounts_sold_t = m.addVars(data[0], data[1], lb=0, vtype=GRB.CONTINUOUS)
p_production_t = m.addVars(data[0],data[1],lb=0, vtype=GRB.CONTINUOUS)
p_holding_t = m.addVars(data[0],data[1],lb=0, vtype=GRB.CONTINUOUS)
p_share = m.addVars(data[1],lb=0, ub=1, vtype=GRB.CONTINUOUS) 
k_share_competition = m.addVars(data[18],lb=0,ub=1, vtype=GRB.CONTINUOUS)
p_slack= m.addVars(data[1],lb=0,ub=1000,vtype=GRB.CONTINUOUS)
#constraints
c1=c2=c3=c4=c5=c6=c7=c8=c9 = list()

for i in range(0, data[0]):
    #soft - capacity
    c8.append(m.addConstr(gp.quicksum(data[12][fam]*product_selected_family[i][fam] for fam in range(0,data[2])) + gp.quicksum(data[8][prod]*product_selected[i][prod] + p_production_t[i,prod]*data[6][prod] for prod in range(0,data[1])) <= data[10][i]))
    for j in range(0, data[1]):
        c1.append( m.addConstr(p_amounts_sold_t[i,j] - (p_share[j])* category_demand_per_period[i][p_belongs_k(j)] <= 0 ))
        c7.append( m.addConstr(p_production_t[i,j] - 99999999999999 * product_selected[i][j] <= 0 ))
        #c8.append( m.addConstr(p_holding_t[i,j] - 9999999999999999 * holding_selected[i][j] <= 0 ))
        if i!=0:
            c6.append(m.addConstr(p_amounts_sold_t[i,j] == p_production_t[i,j] + p_holding_t[i-1,j] - p_holding_t[i,j]))
        else:
            c6.append(m.addConstr(p_amounts_sold_t[i,j] == p_production_t[i,j] - p_holding_t[i,j]))

#3.3 c2
for j in range(0, data[1]):
    c2.append(m.addConstr(p_share[j] <= product_selection[j]))
    c9.append(m.addConstr(p_share[j] >= product_selection[j]*0.000000000000000001))

#3.4 c3
#3.5 c4
#3.6 c5
for k in range(0,len(data[16])):
    array = np.array(data[16][k])
    c5.append(m.addConstr(k_share_competition[k] + gp.quicksum(p_share[aux] for aux in data[16][k]) == 1 ))
    for j in range(0,len(array)):
        c3.append(m.addConstr(p_slack[array[j]] == data[17][array[j]] * k_share_competition[k] - p_share[array[j]] * data[11][k]))
        c4.append(m.addConstr(p_slack[array[j]] <= data[17][array[j]] * k_share_competition[k]) )

#objective function
R = gp.quicksum(data[3][product] * p_amounts_sold_t[period,product] for period in range(0,data[0]) for product in range(0,data[1]))
C1 = gp.quicksum(data[4][product] * p_holding_t[period,product] for period in range(0,data[0]) for product in range(0,data[1]))
C2 = sum(product_selected.sum(axis = 'columns') * data[5])
C3 = sum(product_selected_family.sum(axis = 'columns') * data[13])
C4 = sum(product_selection * data[7])
obj = R - C1 - C2 - C3 - C4
m.setObjective(obj, GRB.MAXIMIZE)
# Suppress Gurobi output by setting the LogToConsole parameter to 0
m.params.LogToConsole = 0
m.optimize()

for j_match in range(0,data[1]):
    for t_match in range(0,data[0]):
        amounts_sold[t_match][j_match] = p_amounts_sold_t[t_match,j_match].X
        production[t_match][j_match] = p_production_t[t_match,j_match].X     
        holding_inventory[t_match][j_match] = p_holding_t[t_match,j_match].X     
    share_product[j_match] = p_share[j_match].X
    slack_product[j_match] = p_slack[j_match].X

for k_match in range(0,K):     
    share_competition[k_match] = k_share_competition[k_match].X
    

verify_feasibility(amounts_sold, share_product, product_selection, share_competition, product_selected, production, product_selected_family, slack_product)
evaluate_function(amounts_sold, holding_inventory, product_selected, product_selected_family, product_selection)
current_evaluation = obj.getValue()
obj.getValue()

### Random but weak

In [ ]:
data[16]

In [ ]:
product_selection[0] = 1
product_selection[2] = 1
product_selected[0][2] = 1
product_selected[1][2] = 1
product_selected[2][2] = 1
product_selected[3][2] = 1
product_selected[4][2] = 1
product_selected[5][2] = 1
product_selected[6][2] = 1
product_selected[7][2] = 1
product_selected[8][2] = 1
product_selected[9][2] = 1
product_selected[10][2] = 1
product_selected[11][2] = 1
product_selected_family[0][0] = 1
product_selected_family[1][0] = 1
product_selected_family[2][0] = 1
product_selected_family[3][0] = 1
product_selected_family[4][0] = 1
product_selected_family[5][0] = 1
product_selected_family[6][0] = 1
product_selected_family[7][0] = 1
product_selected_family[8][0] = 1
product_selected_family[9][0] = 1
product_selected_family[10][1] = 1
product_selected_family[11][1] = 1
product_selected_family[0][1] = 1
product_selected_family[1][1] = 1
product_selected_family[2][1] = 1
product_selected_family[3][1] = 1
product_selected_family[4][1] = 1
product_selected_family[5][1] = 1
product_selected_family[6][1] = 1
product_selected_family[7][1] = 1
product_selected_family[8][1] = 1
product_selected_family[9][1] = 1
product_selected_family[10][1] = 1
product_selected_family[11][1] = 1
product_selected[0][0] = 1
product_selected[1][0] = 1
product_selected[2][0] = 1
product_selected[3][0] = 1
product_selected[4][0] = 1
product_selected[5][0] = 1
product_selected[6][0] = 1
product_selected[7][0] = 1
product_selected[8][0] = 1
product_selected[9][0] = 1
product_selected[10][0] = 1
product_selected[11][0] = 1

m= gp.Model('PLS')
#Decision Variables
p_amounts_sold_t = m.addVars(data[0], data[1], lb=0, vtype=GRB.CONTINUOUS)
p_production_t = m.addVars(data[0],data[1],lb=0, vtype=GRB.CONTINUOUS)
p_holding_t = m.addVars(data[0],data[1],lb=0, vtype=GRB.CONTINUOUS)
p_share = m.addVars(data[1],lb=0, ub=1, vtype=GRB.CONTINUOUS) 
k_share_competition = m.addVars(data[18],lb=0,ub=1, vtype=GRB.CONTINUOUS)
p_slack= m.addVars(data[1],lb=0,ub=1000,vtype=GRB.CONTINUOUS)
#constraints
c1=c2=c3=c4=c5=c6=c7=c8 = list()

for i in range(0, data[0]):
    #soft - capacity
    c8.append(m.addConstr(gp.quicksum(data[12][fam]*product_selected_family[i][fam] for fam in range(0,data[2])) + gp.quicksum(data[8][prod]*product_selected[i][prod] + p_production_t[i,prod]*data[6][prod] for prod in range(0,data[1])) <= data[10][i]))
    for j in range(0, data[1]):
        c1.append( m.addConstr(p_amounts_sold_t[i,j] - (p_share[j])* category_demand_per_period[i][p_belongs_k(j)] <= 0 ))
        c7.append( m.addConstr(p_production_t[i,j] - 99999999999999 * product_selected[i][j] <= 0 ))
        #c8.append( m.addConstr(p_holding_t[i,j] - 9999999999999999 * holding_selected[i][j] <= 0 ))
        if i!=0:
            c6.append(m.addConstr(p_amounts_sold_t[i,j] == p_production_t[i,j] + p_holding_t[i-1,j] - p_holding_t[i,j]))
        else:
            c6.append(m.addConstr(p_amounts_sold_t[i,j] == p_production_t[i,j] - p_holding_t[i,j]))

#3.3 c2
for j in range(0, data[1]):
    c2.append(m.addConstr(p_share[j] <= product_selection[j]))
    c9.append(m.addConstr(p_share[j] >= product_selection[j]*0.000000000000000001))

#3.4 c3
#3.5 c4
#3.6 c5
for k in range(0,len(data[16])):
    array = np.array(data[16][k])
    c5.append(m.addConstr(k_share_competition[k] + gp.quicksum(p_share[aux] for aux in data[16][k]) == 1 ))
    for j in range(0,len(array)):
        c3.append(m.addConstr(p_slack[array[j]] == data[17][array[j]] * k_share_competition[k] - p_share[array[j]] * data[11][k]))
        c4.append(m.addConstr(p_slack[array[j]] <= data[17][array[j]] * k_share_competition[k]) )

#objective function
R = gp.quicksum(data[3][product] * p_amounts_sold_t[period,product] for period in range(0,data[0]) for product in range(0,data[1]))
C1 = gp.quicksum(data[4][product] * p_holding_t[period,product] for period in range(0,data[0]) for product in range(0,data[1]))
C2 = sum(product_selected.sum(axis = 'columns') * data[5])
C3 = sum(product_selected_family.sum(axis = 'columns') * data[13])
C4 = sum(product_selection * data[7])
obj = R - C1 - C2 - C3 - C4
m.setObjective(obj, GRB.MAXIMIZE)
# Suppress Gurobi output by setting the LogToConsole parameter to 0
m.params.LogToConsole = 0
m.optimize()

for j_match in range(0,data[1]):
    for t_match in range(0,data[0]):
        amounts_sold[t_match][j_match] = p_amounts_sold_t[t_match,j_match].X
        production[t_match][j_match] = p_production_t[t_match,j_match].X     
        holding_inventory[t_match][j_match] = p_holding_t[t_match,j_match].X     
    share_product[j_match] = p_share[j_match].X
    slack_product[j_match] = p_slack[j_match].X

for k_match in range(0,K):     
    share_competition[k_match] = k_share_competition[k_match].X
    
verify_feasibility(amounts_sold, share_product, product_selection, share_competition, product_selected, production, product_selected_family, slack_product)
evaluate_function(amounts_sold, holding_inventory, product_selected, product_selected_family, product_selection)
obj.getValue()

### Greedy with composite

In [ ]:
array_added = []
family_added = []
positive_return = True
max_capacity = T * np.mean(data[10])

while positive_return:
    biggest_weight = 0
    new_weight = 0
    j_of_biggest = -1
    
    # get capacity used
    capacity_used_basis = 0 
    
    product_selection = np.zeros(J)
    product_selection[array_added] = 1
    
    for k in range(0,len(data[16])):
        array = np.array(data[16][k])
        for j in range(0,len(array)):           
            if [array[j]] in array_added:
                #get family
                for m_match in range(0,len(data[15])):
                    array_match = np.array(data[15][m_match])
                    for j_match in range(0,len(array_match)):
                        if array_match[j_match] == array[j]:
                            m = m_match
                            n_products_in_family = len(array_match)
                #get share
                share = share_computation(product_selection, array[j])
                amounts_sold.iloc[array[j],:] = share * category_demand_per_period.loc[k,:]
                if m not in family_added:
                    capacity_used_basis = capacity_used_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]] + data[12][m])
                else:
                    capacity_used_basis = capacity_used_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]])
    
    #for each category
    for k in range(0,len(data[16])):
        array = np.array(data[16][k])
        
        #calculate basis of products selected
        for j in range(0,len(array)):
            numerator_basis = 0
            denominator_basis = 0
            if [array[j]] in array_added:
                #get family
                for m_match in range(0,len(data[15])):
                    array_match = np.array(data[15][m_match])
                    for j_match in range(0,len(array_match)):
                        if array_match[j_match] == array[j]:
                            m = m_match
                            n_products_in_family = len(array_match)
                            
                #get share
                share = share_computation(product_selection, array[j])
                #weight
                amounts_sold.iloc[array[j],:] = share * category_demand_per_period.loc[k,:]
                if m not in family_added:
                    numerator_basis = numerator_basis + sum(amounts_sold.iloc[array[j],:]) * data[3][array[j]] - 12 * (data[5][array[j]] + data[13][m]) - data[7][array[j]]
                    denominator_basis = denominator_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]] + data[12][m])
                else:
                    numerator_basis = numerator_basis + sum(amounts_sold.iloc[array[j],:]) * data[3][array[j]] - 12 * (data[5][array[j]]) - data[7][array[j]]
                    denominator_basis = denominator_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]])
        
        #compute weight of new product and check if better than last
        for j in range(0,len(array)):
            if [array[j]] not in array_added:
                product_selection = np.zeros(J)
                product_selection[array_added] = 1
                product_selection[array[j]] = 1
                #get family
                for m_match in range(0,len(data[15])):
                    array_match = np.array(data[15][m_match])
                    for j_match in range(0,len(array_match)):
                        if array_match[j_match] == array[j]:
                            m = m_match
                            n_products_in_family = len(array_match)
                #get share
                share = share_computation(product_selection, array[j])
                #weight
                amounts_sold.iloc[array[j],:] = share * category_demand_per_period.loc[k,:]
                if m not in family_added:
                    numerator = numerator_basis + sum(amounts_sold.iloc[array[j],:]) * data[3][array[j]] - 12 * (data[5][array[j]] + data[13][m]) - data[7][array[j]]
                    denominator = denominator_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]] + data[12][m])
                    capacity_used = capacity_used_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]] + data[12][m])
                else:
                    numerator = numerator_basis + sum(amounts_sold.iloc[array[j],:]) * data[3][array[j]] - 12 * (data[5][array[j]]) - data[7][array[j]]
                    denominator = denominator_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]])
                    capacity_used = capacity_used_basis + sum(amounts_sold.iloc[array[j],:]) * data[8][array[j]] + 12 * (data[6][array[j]])
                new_weight = numerator / denominator
            else:
                new_weight = 0
            
            if (new_weight > biggest_weight) & (capacity_used < max_capacity):
                biggest_weight = new_weight
                j_of_biggest = array[j]
                m_of_biggest = m
    
    #add best
    if j_of_biggest > -1:            
        array_added.append(j_of_biggest)
        if m_of_biggest not in family_added:
            family_added.append(m_of_biggest)           
    else:
        positive_return = False

product_selection = np.zeros(J)
product_selection[array_added] = 1

share_model = gp.Model('Share_optmization')

#decisionvariables
p_share = share_model.addVars(data[1],lb=0, ub=1, vtype=GRB.CONTINUOUS) 
k_share_competition = share_model.addVars(data[18],lb=0,ub=1, vtype=GRB.CONTINUOUS)
p_slack= share_model.addVars(data[1],lb=0,ub=1000000,vtype=GRB.CONTINUOUS)

#constraints
c1=c2=c3=c4=c5 = list()
 
for i in range(0,data[1]):
    c1 = share_model.addConstr(p_share[i]<=product_selection[i])
    c5 = share_model.addConstr(p_share[i]>=product_selection[i]*0.000000000000001)
        
for k in range(0,len(data[16])):
    array = np.array(data[16][k])
    c4.append(share_model.addConstr(k_share_competition[k] + gp.quicksum(p_share[aux] for aux in data[16][k]) == 1 ))
    for j in range(0,len(array)):
        c2.append(share_model.addConstr(p_slack[array[j]] == data[17][array[j]] * k_share_competition[k] - p_share[array[j]] * data[11][k]))
        c3.append(share_model.addConstr(p_slack[array[j]] <= data[17][array[j]] * k_share_competition[k]) )

#objective function
sum_shares = gp.quicksum(p_share[prod] for prod in range(0,data[1])) - gp.quicksum(k_share_competition[comp] for comp in range(0,data[18]))
share_model.setObjective(sum_shares, GRB.MAXIMIZE)
# Suppress Gurobi output by setting the LogToConsole parameter to 0
share_model.params.LogToConsole = 0
share_model.optimize()

for j in range(0,J):
    slack_product[j] = p_slack[j].X
    share_product[j] = p_share[j].X
    
for k in range(0,K):
    share_competition[k] = k_share_competition[k].X
    
#amounts sold computation (substituiu 3.2)
for k in range(0,len(data[16])):
    array = np.array(data[16][k])
    for j in range(0,len(array)):
        amounts_sold.iloc[array[j],:] = share_product[array[j]] * category_demand_per_period.loc[k,:]
        
original_amounts_sold = amounts_sold.copy()

for t in range(T-1,0,-1):
    holding_array = []
    for j in array_added:
        holding_array.append(data[4][j])
    production_planning = amounts_sold.iloc[array_added,:]
    for j in range(0,len(array_added)):
        production_planning.iloc[j,:] = production_planning.iloc[j,:] * holding_array[j]
    sorted_ = production_planning[t].sort_values(ascending = False)
    array_not_added_production = []
    capacity_used = 0
    current_family_added = []
    capacity_used = 0
    for j in range(0,len(sorted_)): 
        temp = 0
        for m_match in range(0,len(data[15])):
                array_match = np.array(data[15][m_match])
                for j_match in range(0,len(array_match)):
                    if array_match[j_match] == np.array(sorted_.index)[j]:
                        m = m_match
        if (m not in current_family_added) & (m in family_added):
            current_family_added.append(m)
            temp = data[12][m]    
        temp = temp + amounts_sold.iloc[np.array(sorted_.index)[j],t] * data[8][np.array(sorted_.index)[j]] + (data[6][np.array(sorted_.index)[j]])
        if (capacity_used + temp) > data[10][t]:
            array_not_added_production.append(np.array(sorted_.index)[j])
        else:
            capacity_used = capacity_used + temp
    
    final_to_add = -9999999999999999999999
    amount_holding = 0
    weight_selected = -9999999999999999999999
    
    for j in array_not_added_production:
        capacity_remaining = data[10][t] - capacity_used
        for m_match in range(0,len(data[15])):
                array_match = np.array(data[15][m_match])
                for j_match in range(0,len(array_match)):
                    if array_match[j_match] == j:
                        m = m_match
        if (m not in current_family_added) & (m in family_added):
            amount_that_fits = (capacity_remaining - (data[6][j] + data[12][m])) / data[8][j]
        else:
            amount_that_fits = (capacity_remaining - (data[6][j]) ) / data[8][j]
            
        if amount_that_fits > 0:
            amount_that_fits = 0
        
        amount_excess = (amounts_sold.iloc[j,t] - amount_that_fits)
        weight = amount_excess * data[4][j] - data[5][j]
        #holding cost * excess amount - setup cost
        if (weight > weight_selected) & (amount_that_fits > 0):
            final_to_add = j
            amount_holding = amount_excess
        
    for j in array_not_added_production:
        if j == final_to_add:
            amounts_sold.iloc[j,t] = amounts_sold.iloc[j,t] - amount_holding
            amounts_sold.iloc[j,t-1] = amounts_sold.iloc[j,t-1] + amount_holding
        else:
            amounts_sold.iloc[j,t-1] = amounts_sold.iloc[j,t-1] + amounts_sold.iloc[j,t]
            amounts_sold.iloc[j,t] = 0

production = amounts_sold.copy()
amounts_sold = original_amounts_sold.copy()

#product selected period computation
product_selected = (production > 0) * 1

#binary setup famility computation
for m in range(0,len(data[15])):
    product_selected_family.iloc[m,:] = product_selected.iloc[data[15][m]].sum()
    product_selected_family = (product_selected_family > 0) * 1
    
#holding inventory computation
holding_inventory.iloc[:,T-1] = 0
for t in range(T-2,-1,-1):
    holding_inventory.iloc[:,t] = amounts_sold.iloc[:,t+1] - production.iloc[:,t+1] + holding_inventory.iloc[:,t+1]

for j in range(0,holding_inventory.shape[0]):
    for t in range(0,holding_inventory.shape[1]):
        if holding_inventory.iloc[j,t] < 0.01:
            holding_inventory.iloc[j,t] = 0
            
for t in range(T-1,0,-1):
    capacity_used = 0
    for j in array_added:
        capacity_used = capacity_used + production.iloc[j,t] * data[8][j] + (data[6][j]) * product_selected.iloc[j,t]
    
    for m in family_added:
        capacity_used = capacity_used + data[12][m] * product_selected_family.iloc[m,t]
        
    if capacity_used > data[10][t]:
        diff = capacity_used - data[10][t]
        print(t)
        print(diff)

BESTamounts_sold = amounts_sold
BESTholding_inventory = holding_inventory
BESTproduct_selection = product_selection
BESTproduct_selected_family = product_selected_family
BESTproduct_selected = product_selected
BESTshare_product = share_product
BESTshare_competition = share_competition
BESTslack_product = slack_product
BESTproduction = production
feasible_solution = verify_feasibility(BESTamounts_sold, BESTshare_product, BESTproduct_selection, BESTshare_competition, BESTproduct_selected, BESTproduction, BESTproduct_selected_family, BESTslack_product)
current_evaluation = evaluate_function(BESTamounts_sold, BESTholding_inventory, BESTproduct_selected, BESTproduct_selected_family, BESTproduct_selection)

m= gp.Model('PLS')
#Decision Variables
p_amounts_sold_t = m.addVars(data[0], data[1], lb=0, vtype=GRB.CONTINUOUS)
p_production_t = m.addVars(data[0],data[1],lb=0, vtype=GRB.CONTINUOUS)
p_holding_t = m.addVars(data[0],data[1],lb=0, vtype=GRB.CONTINUOUS)
p_share = m.addVars(data[1],lb=0, ub=1, vtype=GRB.CONTINUOUS) 
k_share_competition = m.addVars(data[18],lb=0,ub=1, vtype=GRB.CONTINUOUS)
p_slack= m.addVars(data[1],lb=0,ub=1000,vtype=GRB.CONTINUOUS)
#constraints
c1=c2=c3=c4=c5=c6=c7=c8=c9 = list()

for i in range(0, data[0]):
    #soft - capacity
    c8.append(m.addConstr(gp.quicksum(data[12][fam]*product_selected_family[i][fam] for fam in range(0,data[2])) + gp.quicksum(data[8][prod]*product_selected[i][prod] + p_production_t[i,prod]*data[6][prod] for prod in range(0,data[1])) <= data[10][i]))
    for j in range(0, data[1]):
        c1.append( m.addConstr(p_amounts_sold_t[i,j] - (p_share[j])* category_demand_per_period[i][p_belongs_k(j)] <= 0 ))
        c7.append( m.addConstr(p_production_t[i,j] - 99999999999999 * product_selected[i][j] <= 0 ))
        #c8.append( m.addConstr(p_holding_t[i,j] - 9999999999999999 * holding_selected[i][j] <= 0 ))
        if i!=0:
            c6.append(m.addConstr(p_amounts_sold_t[i,j] == p_production_t[i,j] + p_holding_t[i-1,j] - p_holding_t[i,j]))
        else:
            c6.append(m.addConstr(p_amounts_sold_t[i,j] == p_production_t[i,j] - p_holding_t[i,j]))

#3.3 c2
for j in range(0, data[1]):
    c2.append(m.addConstr(p_share[j] <= product_selection[j]))
    c9.append(m.addConstr(p_share[j] >= product_selection[j]*0.000000000000000001))

#3.4 c3
#3.5 c4
#3.6 c5
for k in range(0,len(data[16])):
    array = np.array(data[16][k])
    c5.append(m.addConstr(k_share_competition[k] + gp.quicksum(p_share[aux] for aux in data[16][k]) == 1 ))
    for j in range(0,len(array)):
        c3.append(m.addConstr(p_slack[array[j]] == data[17][array[j]] * k_share_competition[k] - p_share[array[j]] * data[11][k]))
        c4.append(m.addConstr(p_slack[array[j]] <= data[17][array[j]] * k_share_competition[k]) )

#objective function
R = gp.quicksum(data[3][product] * p_amounts_sold_t[period,product] for period in range(0,data[0]) for product in range(0,data[1]))
C1 = gp.quicksum(data[4][product] * p_holding_t[period,product] for period in range(0,data[0]) for product in range(0,data[1]))
C2 = sum(product_selected.sum(axis = 'columns') * data[5])
C3 = sum(product_selected_family.sum(axis = 'columns') * data[13])
C4 = sum(product_selection * data[7])
obj = R - C1 - C2 - C3 - C4
m.setObjective(obj, GRB.MAXIMIZE)
# Suppress Gurobi output by setting the LogToConsole parameter to 0
m.params.LogToConsole = 0
m.optimize()

for j_match in range(0,data[1]):
    for t_match in range(0,data[0]):
        amounts_sold[t_match][j_match] = p_amounts_sold_t[t_match,j_match].X
        production[t_match][j_match] = p_production_t[t_match,j_match].X     
        holding_inventory[t_match][j_match] = p_holding_t[t_match,j_match].X     
    share_product[j_match] = p_share[j_match].X
    slack_product[j_match] = p_slack[j_match].X

for k_match in range(0,K):     
    share_competition[k_match] = k_share_competition[k_match].X
    

verify_feasibility(amounts_sold, share_product, product_selection, share_competition, product_selected, production, product_selected_family, slack_product)
evaluate_function(amounts_sold, holding_inventory, product_selected, product_selected_family, product_selection)
current_evaluation = obj.getValue()
obj.getValue()

## Improvement

In [ ]:
local = local_search_guided(product_selected, current_evaluation)
local[2][-1]

In [ ]:
local_2 = local_search_blind(product_selected, current_evaluation)
local_2[2][-1]

In [ ]:
iterated_local_search_blind(product_selected, current_evaluation, 100)

## Hyperparameter tuning

In [ ]:
temperature = 1000
#cooling_rate = 0.005
cooling_rate = 0.005
cooling_rate = 0.01 #less than 0.15 is suggested in literature, but 0.01 is common practice
cooling_rate = 0.05
cooling_frequency = 5 #literature says between 5 and 10. experiments showed it is not worth it 
temperature_limit = 1 #0.1 is suggested in literature but 1 is enough

In [ ]:
sa_2 = simulated_annealing_blind(product_selected, current_evaluation, temperature, cooling_rate, cooling_frequency, temperature_limit)
sa[1][-1]

In [ ]:
iterated_simulated_annealing_blind(product_selected, current_evaluation, 100, temperature, cooling_rate, cooling_frequency, temperature_limit)